# imports

In [36]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import BPR
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer

import pandas as pd

# global variables

In [3]:
K = 3
VALID_METRIC = 'Recall@'+str(K)
MODEL = 'BPR'
SEED = 2020
USE_GPU = False
SHUFFLE = False 
SHOW_PROGRESS = False

In [3]:
FILENAME = 'sudden_drift_dataset_4000x7_0.71'
SAVE_PATH = 'processed_datasets/artificial_data/'

# functions

In [28]:
def get_filepath(base_folderpath, filename, pt):
    return base_folderpath+filename+pt+'/'+filename+pt

In [5]:
def setup_config_and_dataset(model_name,
                            dataset_name,
                            parameter_dict):
    # configurations initialization
    config = Config(model=model_name, dataset=dataset_name, config_dict=parameter_dict)

    # init random seed
    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    return config, logger, dataset, train_data, valid_data, test_data



def resume_training_from_checkpoint_test(model_name,
                                         dataset_name,
                                         parameter_dict,
                                         checkpoint_file):

    config,\
        logger,\
             dataset,\
                 train_data, valid_data, test_data = setup_config_and_dataset(model_name,
                                                                              dataset_name,
                                                                              parameter_dict)

    # model loading and initialization
    model = BPR(config, train_data.dataset).to(config['device'])
    logger.info(model)

    # trainer loading and initialization
    trainer = Trainer(config, model)
    
    # resume from break point
    # checkpoint_file is the file used to store the model.
    checkpoint_file = dataset_name+'_checkpoint.pth'
    # Load the model parameters information and training information.
    trainer.resume_checkpoint(checkpoint_file)

    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    print('\n\nTraining best results')
    print('best_valid_score: ', best_valid_score)
    print('best_valid_result: ', best_valid_result)

    # model evaluation
    test_result = trainer.evaluate(test_data)
    
    print('\n\nTest results')
    print(test_result)



def test_from_checkpoint(model_name,
                         dataset_name,
                         parameter_dict,
                         checkpoint_file):

    config,\
        logger,\
             dataset,\
                 train_data, valid_data, test_data = setup_config_and_dataset(model_name,
                                                                              dataset_name,
                                                                              parameter_dict)

    # model loading and initialization
    model = BPR(config, train_data.dataset).to(config['device'])
    logger.info(model)


    # trainer loading and initialization
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

    # When calculate ItemCoverage metrics, we need to run this code for set item_nums in eval_collector.
    trainer.eval_collector.data_collect(train_data)

    # model evaluation
    test_result = trainer.evaluate(test_data, model_file=checkpoint_file)

    print(test_result)

# Runtime error solved in later cell

In [ ]:
# Current model
base_dataset_name = 'sudden_drift_dataset_4000x7_0.71'
dataset_name = base_dataset_name+'_pt3'
data_path = 'processed_datasets/artificial_data/'


# Checkpoint - 
checkpoint_ver = 'BPR-Jan-01-2025_16-20-38'
checkpoint_dir = data_path+base_dataset_name+'_pt2'
checkpoint_file = checkpoint_dir+'/'+checkpoint_ver+'.pth'
# checkpoint_file = checkpoint_ver+'.pth'


parameter_dict = {
    'dataset': dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':checkpoint_dir,
    'seed':SEED,
    'shuffle': SHUFFLE
}

test_from_checkpoint(MODEL, dataset_name, parameter_dict, checkpoint_file)

## test_on_old_data

In [4]:
def test_on_old_data(model_name,
                         earlier_dataset_name,
                         current_dataset_name,
                         parameter_dict,
                         checkpoint_file):

    earlier_config,\
        earlier_logger,\
                earlier_dataset,\
                    earlier_train_data, earlier_valid_data, earlier_test_data = setup_config_and_dataset(model_name,
                                                                                earlier_dataset_name,
                                                                                parameter_dict)


    current_config,\
        current_logger,\
                current_dataset,\
                    current_train_data, current_valid_data, current_test_data = setup_config_and_dataset(model_name,
                                                                                current_dataset_name,
                                                                                parameter_dict)

    # model loading and initialization
    current_model = BPR(current_config, current_train_data.dataset).to(current_config['device'])
    current_logger.info(current_model)


    # trainer loading and initialization
    trainer = get_trainer(current_config['MODEL_TYPE'], current_config['model'])(current_config, current_model)

    # When calculate ItemCoverage metrics, we need to run this code for set item_nums in eval_collector.
    trainer.eval_collector.data_collect(earlier_train_data)

    # model evaluation
    test_result = trainer.evaluate(earlier_test_data, model_file=checkpoint_file)

    print(test_result)

In [ ]:
# Current model
base_dataset_name = 'sudden_drift_dataset_4000x7_0.71'
dataset_name = base_dataset_name+'_pt2'
data_path = 'processed_datasets/artificial_data/'


# Checkpoint - 
checkpoint_ver = 'BPR-Jan-01-2025_16-20-40'
checkpoint_dir = data_path+base_dataset_name+'_pt3'
checkpoint_file = checkpoint_dir+'/'+checkpoint_ver+'.pth'
# checkpoint_file = checkpoint_ver+'.pth'



earlier_dataset_name = base_dataset_name+'_pt2'
current_dataset_name = base_dataset_name+'_pt3'


parameter_dict = {
    'dataset': current_dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':checkpoint_dir,
    'seed':SEED,
    'shuffle': SHUFFLE
}

test_on_old_data(MODEL,
                 earlier_dataset_name,
                 current_dataset_name,
                 parameter_dict, checkpoint_file)

# Currently working cell

## test_on_earlier_data 

In [22]:
def test_on_earlier_data(model_name,
                         earlier_datasets,
                         current_dataset_name,
                         parameter_dict,
                         checkpoint_file):


    current_config,\
        current_logger,\
                current_dataset,\
                    current_train_data, current_valid_data, current_test_data = setup_config_and_dataset(model_name,
                                                                                current_dataset_name,
                                                                                parameter_dict)

    # model loading and initialization
    current_model = BPR(current_config, current_train_data.dataset).to(current_config['device'])
    current_logger.info(current_model)


    # trainer loading and initialization
    trainer = get_trainer(current_config['MODEL_TYPE'], current_config['model'])(current_config, current_model)


    # results = []

    for earlier_dataset_name in earlier_datasets:
        print('\n\n'+earlier_dataset_name)
        earlier_config,\
        earlier_logger,\
                earlier_dataset,\
                    earlier_train_data, earlier_valid_data, earlier_test_data = setup_config_and_dataset(model_name,
                                                                                earlier_dataset_name,
                                                                                parameter_dict)


        # When calculate ItemCoverage metrics, we need to run this code for set item_nums in eval_collector.
        # trainer.eval_collector.data_collect(earlier_train_data)
        trainer.eval_collector.data_collect(current_train_data)

        # model evaluation
        test_result = trainer.evaluate(earlier_test_data, model_file=checkpoint_file)
        # results += [test_result]
    
        print(test_result)

In [ ]:
# Current model
base_dataset_name = 'sudden_drift_dataset_4000x7_0.71'
data_path = 'processed_datasets/artificial_data/'

current_ver = '_pt3'
current_dataset_name = base_dataset_name+current_ver

earlier_datasets = [base_dataset_name+'_pt1', base_dataset_name+'_pt8'] # 1,5,6,7,8


# Checkpoint - 
checkpoint_ver = 'BPR-Jan-01-2025_16-20-40'
checkpoint_dir = data_path+base_dataset_name+current_ver
checkpoint_file = checkpoint_dir+'/'+checkpoint_ver+'.pth'





# Suppose we want to leverage random ordering, 
# ratio-based splitting and 
# full ranking with all item candidates, 
# the splitting ratio is set as 8:1:1



parameter_dict = {
    'dataset': current_dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'user_inter_num_interval':'[1,inf)',
    'checkpoint_dir':checkpoint_dir,
    'seed':SEED,
    'shuffle': SHUFFLE,
    # https://recbole.io/docs/user_guide/config/evaluation_settings.html
    'eval_args': {'split': {'LS': 'test_only'},
                  'group_by': 'user',
                  'order': 'RO',
                  'mode': 'pop001'}
}

test_on_earlier_data(MODEL,
                 earlier_datasets,
                 current_dataset_name,
                 parameter_dict, 
                 checkpoint_file)


data_path = processed_datasets/artificial_data/sudden_drift_dataset_4000x7_0.71_pt1
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_4000x7_0.71_pt3
01 Jan 19:05    INFO  Loading model structure and parameters from processed_datasets/artificial_data/sudden_drift_dataset_4000x7_0.71_pt3/BPR-Jan-01-2025_16-20-40.pth

OrderedDict([('recall@3', 0.7214), ('mrr@3', 0.6026), ('ndcg@3', 0.633), ('hit@3', 0.7214), ('precision@3', 0.2405)])


data_path = processed_datasets/artificial_data/sudden_drift_dataset_4000x7_0.71_pt2
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_4000x7_0.71_pt3
01 Jan 19:05    INFO  Loading model structure and parameters from processed_datasets/artificial_data/sudden_drift_dataset_4000x7_0.71_pt3/BPR-Jan-01-2025_16-20-40.pth

OrderedDict([('recall@3', 0.7414), ('mrr@3', 0.6298), ('ndcg@3', 0.6583), ('hit@3', 0.7414), ('precision@3', 0.2471)])

# isolate user seen all items error trigger conditions


in these tests, the user 0 was added only at the beginning

In [7]:
# Current model
base_dataset_name = 'sudden_drift_dataset_4000x7_0.71'
data_path = 'processed_datasets/artificial_data/'

In [23]:
def trigger_error(data_path, base_dataset_name, model_ver, checkpoint_ver, data_ver):
    current_ver = model_ver
    current_dataset_name = base_dataset_name+current_ver

    earlier_datasets = [base_dataset_name+data_ver]#,base_dataset_name+'_pt6', base_dataset_name+'_pt7', base_dataset_name+'_pt8'] # 1,5,6,7,8


    # Checkpoint - 
    # checkpoint_ver = 'BPR-Jan-01-2025_16-20-32'
    checkpoint_dir = data_path+base_dataset_name+current_ver
    checkpoint_file = checkpoint_dir+'/'+checkpoint_ver+'.pth'


    parameter_dict = {
        'dataset': current_dataset_name+'.inter',
        'data_path': data_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        'valid_metric':VALID_METRIC,
        'user_inter_num_interval':'[1,inf)',
        'checkpoint_dir':checkpoint_dir,
        'seed':SEED,
        'shuffle': SHUFFLE,
        'state':'ERROR',
        # 'show_progress': SHOW_PROGRESS,
        # https://recbole.io/docs/user_guide/config/evaluation_settings.html
        'eval_args': {'split': {'LS': 'test_only'}, # leave-one-out sample type
                    'group_by': 'user',
                    'order': 'RO', # random ordering
                    'mode': 'pop001'} #  for example pop100, means sample 100 negative items for each positive item in testing set based on item popularity (Counter(item) in .inter file), and evaluate the model on these positive items with their sampled negative items
    }

    test_on_earlier_data(MODEL,
                    earlier_datasets,
                    current_dataset_name,
                    parameter_dict, 
                    checkpoint_file)

## Model pt1

### model pt1 vs data pt5 : user seen all items error (UsI Error)

ValueError: Some users have interacted with all items, which we can not sample negative items for them. Please set `user_inter_num_interval` to filter those users.

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-01-2025_16-20-32', data_ver='_pt5')

### model pt1 vs data pt6 : (UsI Error)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-01-2025_16-20-32', data_ver='_pt6')

### trigger_error(model_ver='_pt1', data_ver='_pt7') (IndexError: index out of range in self)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-01-2025_16-20-32', data_ver='_pt7')

### trigger_error(model_ver='_pt1', data_ver='_pt8') (UsI error)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-01-2025_16-20-32', data_ver='_pt8')

## Model pt2

### trigger_error(model_ver='_pt2', data_ver='_pt1'): pass

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-01-2025_16-20-38', data_ver='_pt1')

### trigger_error(model_ver='_pt2', data_ver='_pt5'): (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-01-2025_16-20-38', data_ver='_pt5')

### trigger_error(model_ver='_pt2', data_ver='_pt6'): (UsI error)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-01-2025_16-20-38', data_ver='_pt6')

### trigger_error(model_ver='_pt2', data_ver='_pt7'): (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-01-2025_16-20-38', data_ver='_pt7')

### trigger_error(model_ver='_pt2', data_ver='_pt8'): (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-01-2025_16-20-38', data_ver='_pt8')

## Model pt3

### trigger_error(model_ver='_pt3', data_ver='_pt1'): pass

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-01-2025_16-20-40', data_ver='_pt1')

### trigger_error(model_ver='_pt3', data_ver='_pt2'): pass

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-01-2025_16-20-40', data_ver='_pt2')

### trigger_error(model_ver='_pt3',  data_ver='_pt5'): (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-01-2025_16-20-40', data_ver='_pt5')

### trigger_error(model_ver='_pt3', data_ver='_pt6'): (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-01-2025_16-20-40', data_ver='_pt6')

### trigger_error(model_ver='_pt3', data_ver='_pt7'): (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-01-2025_16-20-40', data_ver='_pt7')

### trigger_error(model_ver='_pt3', data_ver='_pt8'): (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-01-2025_16-20-40', data_ver='_pt8')

## Model full (pt4)

### trigger_error(model_ver='',data_ver='_pt1'): pass

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-01-2025_16-20-44', data_ver='_pt1')

### trigger_error(model_ver='', data_ver='_pt2'): pass

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-01-2025_16-20-44', data_ver='_pt2')

### trigger_error(model_ver='', data_ver='_pt3'): pass

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-01-2025_16-20-44', data_ver='_pt3')

### trigger_error(model_ver='', data_ver='_pt5'): (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-01-2025_16-20-44', data_ver='_pt5')

### trigger_error(model_ver='', data_ver='_pt6'): : (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-01-2025_16-20-44', data_ver='_pt6')

### trigger_error(model_ver='', data_ver='_pt7'): pass

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-01-2025_16-20-44', data_ver='_pt7')

### trigger_error(model_ver='', data_ver='_pt8'): : (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-01-2025_16-20-44', data_ver='_pt8')

# what items does pt5 does not have?

In [1]:
import pandas as pd

In [2]:
def get_filepath(base_folderpath, filename, pt):
    return base_folderpath+filename+pt+'/'+filename+pt

In [3]:
filename = 'sudden_drift_dataset_4000x7_0.71'
save_path = 'processed_datasets/artificial_data/'

In [ ]:
df_pt1 = pd.read_csv(get_filepath(save_path, filename, '_pt1')+'.csv')
df_pt1.item_id.value_counts()

In [ ]:
df_pt2 = pd.read_csv(get_filepath(save_path, filename, '_pt2')+'.csv')
df_pt2.item_id.value_counts()

In [ ]:
df_pt3 = pd.read_csv(get_filepath(save_path, filename, '_pt3')+'.csv')
df_pt3.item_id.value_counts()

In [ ]:
df_full = pd.read_csv(get_filepath(save_path, filename, '')+'.csv')
df_full.item_id.value_counts()

In [ ]:
df_full[['user_id','item_id']].groupby('user_id').count().median()

In [ ]:
df_pt5 = pd.read_csv(get_filepath(save_path, filename, '_pt5')+'.csv')
df_pt5.item_id.value_counts()

In [ ]:
df_pt6 = pd.read_csv(get_filepath(save_path, filename, '_pt6')+'.csv')
df_pt6.item_id.value_counts()

# generate artificial datasets

code copied from notebook 04 and altered

In [ ]:
# filename = 'sudden_drift_dataset_4000x7_0.71'
# save_path = 'processed_datasets/artificial_data/'

In [5]:
df_full = pd.read_csv(get_filepath(SAVE_PATH, FILENAME, '')+'.csv')
df_full.item_id.value_counts()

item_id
drifted_i_5    1288
drifted_i_1    1288
i_1            1284
i_5            1284
i_3            1000
i_2            1000
i_4            1000
Name: count, dtype: int64

In [6]:
bin_size = 1000

In [9]:
df_full[:bin_size*3].reset_index(drop=True).item_id.value_counts()

item_id
i_5            946
i_1            940
i_2            382
i_3            366
i_4            364
drifted_i_5      1
drifted_i_1      1
Name: count, dtype: int64

In [10]:
df_full['user_id_n'] = df_full['user_id'].apply(lambda x: x[2:])
df_full['user_id_n'] = df_full['user_id_n'].astype(int)
df_full.sort_values(by='user_id_n', inplace=True)
df_full.reset_index(drop=True, inplace=True)

df_full[:bin_size*3].reset_index(drop=True).item_id.value_counts()

item_id
i_5            946
i_1            940
i_2            382
i_3            366
i_4            364
drifted_i_5      1
drifted_i_1      1
Name: count, dtype: int64

In [11]:
df_full

,user_id,item_id,timestamp,user_id_n
0,u_0,drifted_i_5,1.735093e+09,0
1,u_0,drifted_i_1,1.735093e+09,0
2,u_1,i_1,1.735093e+09,1
3,u_2,i_1,1.735093e+09,2
4,u_2,i_3,1.735093e+09,2
...,...,...,...,...
8139,u_3998,drifted_i_1,1.735093e+09,3998
8140,u_3998,drifted_i_5,1.735093e+09,3998
8141,u_3999,i_3,1.735093e+09,3999
8142,u_3999,drifted_i_1,1.735093e+09,3999


In [12]:
n_users = 4000 # bc of random sample, some users dont have occurrencies
n_items = 5
sudden_drift_start = 2000  # Starting user index for drift (1-indexed)
bin_size = 1000

In [13]:
users_list = [f'u_{i+1}' for i in range(n_users)]

In [15]:
users_list[:bin_size]

['u_1',
 'u_2',
 'u_3',
 'u_4',
 'u_5',
 'u_6',
 'u_7',
 'u_8',
 'u_9',
 'u_10',
 'u_11',
 'u_12',
 'u_13',
 'u_14',
 'u_15',
 'u_16',
 'u_17',
 'u_18',
 'u_19',
 'u_20',
 'u_21',
 'u_22',
 'u_23',
 'u_24',
 'u_25',
 'u_26',
 'u_27',
 'u_28',
 'u_29',
 'u_30',
 'u_31',
 'u_32',
 'u_33',
 'u_34',
 'u_35',
 'u_36',
 'u_37',
 'u_38',
 'u_39',
 'u_40',
 'u_41',
 'u_42',
 'u_43',
 'u_44',
 'u_45',
 'u_46',
 'u_47',
 'u_48',
 'u_49',
 'u_50',
 'u_51',
 'u_52',
 'u_53',
 'u_54',
 'u_55',
 'u_56',
 'u_57',
 'u_58',
 'u_59',
 'u_60',
 'u_61',
 'u_62',
 'u_63',
 'u_64',
 'u_65',
 'u_66',
 'u_67',
 'u_68',
 'u_69',
 'u_70',
 'u_71',
 'u_72',
 'u_73',
 'u_74',
 'u_75',
 'u_76',
 'u_77',
 'u_78',
 'u_79',
 'u_80',
 'u_81',
 'u_82',
 'u_83',
 'u_84',
 'u_85',
 'u_86',
 'u_87',
 'u_88',
 'u_89',
 'u_90',
 'u_91',
 'u_92',
 'u_93',
 'u_94',
 'u_95',
 'u_96',
 'u_97',
 'u_98',
 'u_99',
 'u_100',
 'u_101',
 'u_102',
 'u_103',
 'u_104',
 'u_105',
 'u_106',
 'u_107',
 'u_108',
 'u_109',
 'u_110',
 'u_111'

In [16]:
users_list[:bin_size*3]

['u_1',
 'u_2',
 'u_3',
 'u_4',
 'u_5',
 'u_6',
 'u_7',
 'u_8',
 'u_9',
 'u_10',
 'u_11',
 'u_12',
 'u_13',
 'u_14',
 'u_15',
 'u_16',
 'u_17',
 'u_18',
 'u_19',
 'u_20',
 'u_21',
 'u_22',
 'u_23',
 'u_24',
 'u_25',
 'u_26',
 'u_27',
 'u_28',
 'u_29',
 'u_30',
 'u_31',
 'u_32',
 'u_33',
 'u_34',
 'u_35',
 'u_36',
 'u_37',
 'u_38',
 'u_39',
 'u_40',
 'u_41',
 'u_42',
 'u_43',
 'u_44',
 'u_45',
 'u_46',
 'u_47',
 'u_48',
 'u_49',
 'u_50',
 'u_51',
 'u_52',
 'u_53',
 'u_54',
 'u_55',
 'u_56',
 'u_57',
 'u_58',
 'u_59',
 'u_60',
 'u_61',
 'u_62',
 'u_63',
 'u_64',
 'u_65',
 'u_66',
 'u_67',
 'u_68',
 'u_69',
 'u_70',
 'u_71',
 'u_72',
 'u_73',
 'u_74',
 'u_75',
 'u_76',
 'u_77',
 'u_78',
 'u_79',
 'u_80',
 'u_81',
 'u_82',
 'u_83',
 'u_84',
 'u_85',
 'u_86',
 'u_87',
 'u_88',
 'u_89',
 'u_90',
 'u_91',
 'u_92',
 'u_93',
 'u_94',
 'u_95',
 'u_96',
 'u_97',
 'u_98',
 'u_99',
 'u_100',
 'u_101',
 'u_102',
 'u_103',
 'u_104',
 'u_105',
 'u_106',
 'u_107',
 'u_108',
 'u_109',
 'u_110',
 'u_111'

In [19]:
df_full[df_full.user_id.isin(users_list[bin_size:bin_size*3])].reset_index(drop=True).item_id.value_counts()

item_id
drifted_i_1    661
drifted_i_5    651
i_5            640
i_1            622
i_3            500
i_4            500
i_2            493
Name: count, dtype: int64

In [32]:
import os
import pandas as pd
import random
from datetime import datetime
import time
import pickle


def validate_folderpath(folderpath):
    # print('im validating')
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)
        print('Folder created: ', folderpath)


def save_picklefile(d, filepath):
    with open(filepath, 'wb') as f:
            pickle.dump(d, f)

    print('Saved file at '+filepath)

def load_picklefile(filepath):
    with open(filepath, 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

    


def generate_artificial_random_dataset(n_users,
                                       n_items, 
                                       ts,
                                       all_items_seen,
                                       random_seed,
                                       n_items_to_drift,
                                       sudden_drift_start,
                                       drift_items_freq_list,
                                       non_drift_items_freq_list,
                                       save_path,
                                       base_filename,
                                       bin_size):
    
    def add_zero_user(df):

        df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
        df['user_id_n'] = df['user_id_n'].astype(int)

        # df.loc[-1] = ['u_-1', 'i_1', ts, 0]
        # df.loc[-2] = ['u_-1', 'i_2', ts, 0]
        # df.loc[-3] = ['u_-1', 'i_3', ts, 0]
        # df.loc[-4] = ['u_-1', 'i_5', ts, 0]
        # df.loc[-5] = ['u_0', 'i_4', ts, 0]
        df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
        df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

        df.sort_values(by='user_id_n', inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.drop(columns=['user_id_n'], inplace=True)

        # print('added zero user\n', df.head())
        return df

    
    def create_folderpath(save_path, base_filename, specs_str):
        return save_path+base_filename+'_'+specs_str+'/'
    

    def save_items_frequencies(n_items_to_drift,
                               sudden_drift_start,
                               drift_items_freq_list,
                               non_drift_items_freq_list,
                               save_path, 
                               base_filename,
                               specs_str):
        
        folderpath = create_folderpath(save_path, base_filename, specs_str)
        validate_folderpath(folderpath)

        d = {'n_items_to_drift': n_items_to_drift,
             'sudden_drift_start': sudden_drift_start,
             'drift_items_freq_list': drift_items_freq_list,
             'non_drift_items_freq_list': non_drift_items_freq_list}

        save_picklefile(d, folderpath+'saved_dictionary.pkl')
       

    
    def save_dataset_atomic_file(df, user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())


    def add_user0_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)
            df_sampled = add_zero_user(df_sampled)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())




    def split_dataset_into_4_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

        
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def split_dataset_into_4_add_user0_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        add_user0_save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

                
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def calculate_sparsity(df):
        # df.item_id.groupby([df.user_id, df.item_id]).count().sum() == df.user_id.count()
        sparsity = 1 - df.user_id.count()/(df.user_id.nunique()*df.item_id.nunique())
        specs_str = str(df.user_id.nunique())+'x'+str(df.item_id.nunique())+'_'+str(round(sparsity, 2))
        print('specs_str', specs_str)
        return sparsity, specs_str
    

    def rename_item(row):
        if int(row['user_id'].split('_')[1]) > sudden_drift_start and row['item_id'] in renamed_items:
            return renamed_items[row['item_id']]
        return row['item_id']




    users_list = [f'u_{i+1}' for i in range(n_users)]
    items_list = [f'i_{j+1}' for j in range(n_items)]


    if all_items_seen:

        data = []
        for user in users_list:
            for item in items_list:
                data.append({'user_id': user, 'item_id': item, 'timestamp':ts})

        all_items_seen_df = pd.DataFrame(data)

        # Introduce sudden drift
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}       
        

        all_items_seen_df['item_id'] = all_items_seen_df.apply(rename_item, axis=1)
        # print(all_items_seen_df.item_id.groupby([all_items_seen_df.user_id, all_items_seen_df.item_id]).count().unstack().fillna(0).astype(int))

        
        sparsity , specs_str = calculate_sparsity(all_items_seen_df)
        print('sparsity: ',sparsity)

        sampled_df = add_zero_user(sampled_df)
        users_list.insert(0, 'u_0')
        

        split_dataset_into_4_and_save_atomic_file(all_items_seen_df,users_list, bin_size, save_path, specs_str)
        # save_dataset_atomic_file(all_items_seen_df, save_path, specs_str)
        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)
        

    else:
        
        if len(drift_items_freq_list) != n_items_to_drift:
            print('Not all items frequency was specified!')
            return None
        elif len(non_drift_items_freq_list) != len(items_list)-n_items_to_drift:
            print('Not all items frequency was specified!')
            return None


        def sample_with_repetition_of_pattern(users_list, items_list, items_freq_list):
            random.seed(random_seed)
            sampled_df = pd.DataFrame({})
            for i, freq in enumerate(items_freq_list):
                # print('k ',k)
                user_sample = random.sample(users_list[:sudden_drift_start], k=freq) +\
                                random.sample(users_list[sudden_drift_start:], k=freq)
                temp_df = pd.DataFrame({'user_id': user_sample, 
                                        'item_id': items_list[i]})
                # print(temp_df.item_id.groupby([temp_df.user_id, temp_df.item_id]).count().unstack().fillna(0).astype(int))
                sampled_df = pd.concat([sampled_df, temp_df])
            
            return sampled_df
        

        # Introduce sudden drift
        # No need to random sample, bc the list will have the frequencies for each item
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        # drift_items_list = [items_list[i] for i,x in enumerate(items_freq_list) if x == sudden_drift_start]
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}
        non_drift_items_list = list(set(items_list) - set(drift_items_list))
        
        print('drift_items_list', drift_items_list)
        print('renamed_items', renamed_items)
        print('non_drift_items_list', non_drift_items_list)

        
        random.seed(random_seed)  # For reproducibility
        sampled_df = sample_with_repetition_of_pattern(users_list,
                                                       non_drift_items_list,
                                                       non_drift_items_freq_list)
        
        sampled_df = pd.concat([sampled_df,
                                sample_with_repetition_of_pattern(users_list,
                                                                    drift_items_list,
                                                                    drift_items_freq_list)])

        if sampled_df.user_id.nunique() < n_users:
            # print(sampled_df.head())
            users_not_sampled = list(set(users_list) - set(sampled_df.user_id))
            print('users_not_sampled', len(users_not_sampled))
            # print('drift_items_list', drift_items_list)
            for user in users_not_sampled:
                for item in drift_items_list:
                    # print(sampled_df.loc[sampled_df['user_id']==user, 'item_id'].count())
                    sampled_df.loc[len(sampled_df)] = [user, item]


        sampled_df['item_id'] = sampled_df.apply(rename_item, axis=1)
        # print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))

        sampled_df['timestamp'] = ts



        sparsity, specs_str = calculate_sparsity(sampled_df)
        print('sparsity: ',sparsity)
        # print(specs_str)
        print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))
        # print(sampled_df.head())


        # when trainning on pt1, yield ValueError: Some users have interacted with all items, which we can not sample negative items for them. Please set `user_inter_num_interval` to filter those users.
        sampled_df = add_zero_user(sampled_df) # to solve the error 
        users_list.insert(0, 'u_0')

        # save_dataset_atomic_file(sampled_df, save_path, specs_str)
        # split_dataset_into_4_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        split_dataset_into_4_add_user0_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)

        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)


        return sampled_df

### corrected data splittage, with user0 added at the beginning (note that the function needs to be edited to run this one 1st)

In [16]:
import os
import pandas as pd
import random
from datetime import datetime
import time
import pickle


def validate_folderpath(folderpath):
    # print('im validating')
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)
        print('Folder created: ', folderpath)


def save_picklefile(d, filepath):
    with open(filepath, 'wb') as f:
            pickle.dump(d, f)

    print('Saved file at '+filepath)

def load_picklefile(filepath):
    with open(filepath, 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

    


def generate_artificial_random_dataset(n_users,
                                       n_items, 
                                       ts,
                                       all_items_seen,
                                       random_seed,
                                       n_items_to_drift,
                                       sudden_drift_start,
                                       drift_items_freq_list,
                                       non_drift_items_freq_list,
                                       save_path,
                                       base_filename,
                                       bin_size):
    
    def add_zero_user(df):

        df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
        df['user_id_n'] = df['user_id_n'].astype(int)

        # df.loc[-1] = ['u_-1', 'i_1', ts, 0]
        # df.loc[-2] = ['u_-1', 'i_2', ts, 0]
        # df.loc[-3] = ['u_-1', 'i_3', ts, 0]
        # df.loc[-4] = ['u_-1', 'i_5', ts, 0]
        # df.loc[-5] = ['u_0', 'i_4', ts, 0]
        df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
        df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

        df.sort_values(by='user_id_n', inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.drop(columns=['user_id_n'], inplace=True)

        # print('added zero user\n', df.head())
        return df

    
    def create_folderpath(save_path, base_filename, specs_str):
        return save_path+base_filename+'_'+specs_str+'/'
    

    def save_items_frequencies(n_items_to_drift,
                               sudden_drift_start,
                               drift_items_freq_list,
                               non_drift_items_freq_list,
                               save_path, 
                               base_filename,
                               specs_str):
        
        folderpath = create_folderpath(save_path, base_filename, specs_str)
        validate_folderpath(folderpath)

        d = {'n_items_to_drift': n_items_to_drift,
             'sudden_drift_start': sudden_drift_start,
             'drift_items_freq_list': drift_items_freq_list,
             'non_drift_items_freq_list': non_drift_items_freq_list}

        save_picklefile(d, folderpath+'saved_dictionary.pkl')
       

    
    def save_dataset_atomic_file(df, user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())


    def add_user0_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)
            df_sampled = add_zero_user(df_sampled)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())




    def split_dataset_into_4_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

        
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def split_dataset_into_4_add_user0_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        add_user0_save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

                
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def calculate_sparsity(df):
        # df.item_id.groupby([df.user_id, df.item_id]).count().sum() == df.user_id.count()
        sparsity = 1 - df.user_id.count()/(df.user_id.nunique()*df.item_id.nunique())
        specs_str = str(df.user_id.nunique())+'x'+str(df.item_id.nunique())+'_'+str(round(sparsity, 2))
        print('specs_str', specs_str)
        return sparsity, specs_str
    

    def rename_item(row):
        if int(row['user_id'].split('_')[1]) > sudden_drift_start and row['item_id'] in renamed_items:
            return renamed_items[row['item_id']]
        return row['item_id']




    users_list = [f'u_{i+1}' for i in range(n_users)]
    items_list = [f'i_{j+1}' for j in range(n_items)]


    if all_items_seen:

        data = []
        for user in users_list:
            for item in items_list:
                data.append({'user_id': user, 'item_id': item, 'timestamp':ts})

        all_items_seen_df = pd.DataFrame(data)

        # Introduce sudden drift
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}       
        

        all_items_seen_df['item_id'] = all_items_seen_df.apply(rename_item, axis=1)
        # print(all_items_seen_df.item_id.groupby([all_items_seen_df.user_id, all_items_seen_df.item_id]).count().unstack().fillna(0).astype(int))

        
        sparsity , specs_str = calculate_sparsity(all_items_seen_df)
        print('sparsity: ',sparsity)

        sampled_df = add_zero_user(sampled_df)
        users_list.insert(0, 'u_0')
        

        split_dataset_into_4_and_save_atomic_file(all_items_seen_df,users_list, bin_size, save_path, specs_str)
        # save_dataset_atomic_file(all_items_seen_df, save_path, specs_str)
        # split_dataset_into_4_add_user0_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)
        

    else:
        
        if len(drift_items_freq_list) != n_items_to_drift:
            print('Not all items frequency was specified!')
            return None
        elif len(non_drift_items_freq_list) != len(items_list)-n_items_to_drift:
            print('Not all items frequency was specified!')
            return None


        def sample_with_repetition_of_pattern(users_list, items_list, items_freq_list):
            random.seed(random_seed)
            sampled_df = pd.DataFrame({})
            for i, freq in enumerate(items_freq_list):
                # print('k ',k)
                user_sample = random.sample(users_list[:sudden_drift_start], k=freq) +\
                                random.sample(users_list[sudden_drift_start:], k=freq)
                temp_df = pd.DataFrame({'user_id': user_sample, 
                                        'item_id': items_list[i]})
                # print(temp_df.item_id.groupby([temp_df.user_id, temp_df.item_id]).count().unstack().fillna(0).astype(int))
                sampled_df = pd.concat([sampled_df, temp_df])
            
            return sampled_df
        

        # Introduce sudden drift
        # No need to random sample, bc the list will have the frequencies for each item
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        # drift_items_list = [items_list[i] for i,x in enumerate(items_freq_list) if x == sudden_drift_start]
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}
        non_drift_items_list = list(set(items_list) - set(drift_items_list))
        
        print('drift_items_list', drift_items_list)
        print('renamed_items', renamed_items)
        print('non_drift_items_list', non_drift_items_list)

        
        random.seed(random_seed)  # For reproducibility
        sampled_df = sample_with_repetition_of_pattern(users_list,
                                                       non_drift_items_list,
                                                       non_drift_items_freq_list)
        
        sampled_df = pd.concat([sampled_df,
                                sample_with_repetition_of_pattern(users_list,
                                                                    drift_items_list,
                                                                    drift_items_freq_list)])

        if sampled_df.user_id.nunique() < n_users:
            # print(sampled_df.head())
            users_not_sampled = list(set(users_list) - set(sampled_df.user_id))
            print('users_not_sampled', len(users_not_sampled))
            # print('drift_items_list', drift_items_list)
            for user in users_not_sampled:
                for item in drift_items_list:
                    # print(sampled_df.loc[sampled_df['user_id']==user, 'item_id'].count())
                    sampled_df.loc[len(sampled_df)] = [user, item]


        sampled_df['item_id'] = sampled_df.apply(rename_item, axis=1)
        # print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))

        sampled_df['timestamp'] = ts



        sparsity, specs_str = calculate_sparsity(sampled_df)
        print('sparsity: ',sparsity)
        # print(specs_str)
        print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))
        # print(sampled_df.head())


        # when trainning on pt1, yield ValueError: Some users have interacted with all items, which we can not sample negative items for them. Please set `user_inter_num_interval` to filter those users.
        sampled_df = add_zero_user(sampled_df) # to solve the error 
        users_list.insert(0, 'u_0')

        # save_dataset_atomic_file(sampled_df, save_path, specs_str)
        split_dataset_into_4_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        # split_dataset_into_4_add_user0_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)

        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)


        return sampled_df

# Parameters
n_users = 4000 # bc of random sample, some users dont have occurrencies
n_items = 5
sudden_drift_start = 2000  # Starting user index for drift (1-indexed)
bin_size = 1000

string = "24/12/2024 21:12:24"
ts = time.mktime(datetime.strptime(string, "%d/%m/%Y %H:%M:%S").timetuple())

all_items_seen = False

random_seed = 42

ratio_to_drift = 2 # Select 50% of items to rename
n_items_to_drift = n_items // ratio_to_drift

base_filename = 'sudden_drift_dataset_updated'
save_path = 'processed_datasets/artificial_data/'

drift_items_freq_list = [sudden_drift_start//2, sudden_drift_start//2]
non_drift_items_freq_list = [sudden_drift_start//4, sudden_drift_start//4, sudden_drift_start//4]

df = generate_artificial_random_dataset(n_users=n_users,
                                    n_items=n_items, 
                                    ts=ts,
                                    all_items_seen=all_items_seen,
                                    n_items_to_drift=n_items_to_drift,
                                    random_seed=random_seed,
                                    sudden_drift_start=sudden_drift_start,
                                    drift_items_freq_list=drift_items_freq_list,
                                    non_drift_items_freq_list=non_drift_items_freq_list,
                                    save_path=save_path,
                                    base_filename=base_filename,
                                    bin_size=bin_size)

drift_items_list ['i_1', 'i_5']
renamed_items {'i_1': 'drifted_i_1', 'i_5': 'drifted_i_5'}
non_drift_items_list ['i_2', 'i_3', 'i_4']
users_not_sampled 571
specs_str 4000x7_0.71
sparsity:  0.7092142857142858
item_id  drifted_i_1  drifted_i_5  i_1  i_2  i_3  i_4  i_5
user_id                                                   
u_1                0            0    1    0    0    0    0
u_10               0            0    1    0    0    0    1
u_100              0            0    0    0    0    0    1
u_1000             0            0    1    0    1    0    0
u_1001             0            0    1    1    1    0    1
...              ...          ...  ...  ...  ...  ...  ...
u_995              0            0    1    1    0    0    0
u_996              0            0    1    1    0    1    0
u_997              0            0    1    1    0    0    0
u_998              0            0    1    0    0    0    1
u_999              0            0    1    0    1    0    0

[4000 rows x 7 columns]


### corrected data splittage, with user0 added in all parts (note that the function needs to be edited to run this one 1st)

In [33]:
import os
import pandas as pd
import random
from datetime import datetime
import time
import pickle


def validate_folderpath(folderpath):
    # print('im validating')
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)
        print('Folder created: ', folderpath)


def save_picklefile(d, filepath):
    with open(filepath, 'wb') as f:
            pickle.dump(d, f)

    print('Saved file at '+filepath)

def load_picklefile(filepath):
    with open(filepath, 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

    


def generate_artificial_random_dataset(n_users,
                                       n_items, 
                                       ts,
                                       all_items_seen,
                                       random_seed,
                                       n_items_to_drift,
                                       sudden_drift_start,
                                       drift_items_freq_list,
                                       non_drift_items_freq_list,
                                       save_path,
                                       base_filename,
                                       bin_size):
    
    def add_zero_user(df):

        df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
        df['user_id_n'] = df['user_id_n'].astype(int)

        # df.loc[-1] = ['u_-1', 'i_1', ts, 0]
        # df.loc[-2] = ['u_-1', 'i_2', ts, 0]
        # df.loc[-3] = ['u_-1', 'i_3', ts, 0]
        # df.loc[-4] = ['u_-1', 'i_5', ts, 0]
        # df.loc[-5] = ['u_0', 'i_4', ts, 0]
        df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
        df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

        df.sort_values(by='user_id_n', inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.drop(columns=['user_id_n'], inplace=True)

        # print('added zero user\n', df.head())
        return df

    
    def create_folderpath(save_path, base_filename, specs_str):
        return save_path+base_filename+'_'+specs_str+'/'
    

    def save_items_frequencies(n_items_to_drift,
                               sudden_drift_start,
                               drift_items_freq_list,
                               non_drift_items_freq_list,
                               save_path, 
                               base_filename,
                               specs_str):
        
        folderpath = create_folderpath(save_path, base_filename, specs_str)
        validate_folderpath(folderpath)

        d = {'n_items_to_drift': n_items_to_drift,
             'sudden_drift_start': sudden_drift_start,
             'drift_items_freq_list': drift_items_freq_list,
             'non_drift_items_freq_list': non_drift_items_freq_list}

        save_picklefile(d, folderpath+'saved_dictionary.pkl')
       

    
    def save_dataset_atomic_file(df, user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())


    def add_user0_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)
            df_sampled = add_zero_user(df_sampled)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())




    def split_dataset_into_4_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

        
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def split_dataset_into_4_add_user0_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        add_user0_save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

                
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def calculate_sparsity(df):
        # df.item_id.groupby([df.user_id, df.item_id]).count().sum() == df.user_id.count()
        sparsity = 1 - df.user_id.count()/(df.user_id.nunique()*df.item_id.nunique())
        specs_str = str(df.user_id.nunique())+'x'+str(df.item_id.nunique())+'_'+str(round(sparsity, 2))
        print('specs_str', specs_str)
        return sparsity, specs_str
    

    def rename_item(row):
        if int(row['user_id'].split('_')[1]) > sudden_drift_start and row['item_id'] in renamed_items:
            return renamed_items[row['item_id']]
        return row['item_id']




    users_list = [f'u_{i+1}' for i in range(n_users)]
    items_list = [f'i_{j+1}' for j in range(n_items)]


    if all_items_seen:

        data = []
        for user in users_list:
            for item in items_list:
                data.append({'user_id': user, 'item_id': item, 'timestamp':ts})

        all_items_seen_df = pd.DataFrame(data)

        # Introduce sudden drift
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}       
        

        all_items_seen_df['item_id'] = all_items_seen_df.apply(rename_item, axis=1)
        # print(all_items_seen_df.item_id.groupby([all_items_seen_df.user_id, all_items_seen_df.item_id]).count().unstack().fillna(0).astype(int))

        
        sparsity , specs_str = calculate_sparsity(all_items_seen_df)
        print('sparsity: ',sparsity)

        sampled_df = add_zero_user(sampled_df)
        users_list.insert(0, 'u_0')
        

        split_dataset_into_4_and_save_atomic_file(all_items_seen_df,users_list, bin_size, save_path, specs_str)
        # save_dataset_atomic_file(all_items_seen_df, save_path, specs_str)
        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)
        

    else:
        
        if len(drift_items_freq_list) != n_items_to_drift:
            print('Not all items frequency was specified!')
            return None
        elif len(non_drift_items_freq_list) != len(items_list)-n_items_to_drift:
            print('Not all items frequency was specified!')
            return None


        def sample_with_repetition_of_pattern(users_list, items_list, items_freq_list):
            random.seed(random_seed)
            sampled_df = pd.DataFrame({})
            for i, freq in enumerate(items_freq_list):
                # print('k ',k)
                user_sample = random.sample(users_list[:sudden_drift_start], k=freq) +\
                                random.sample(users_list[sudden_drift_start:], k=freq)
                temp_df = pd.DataFrame({'user_id': user_sample, 
                                        'item_id': items_list[i]})
                # print(temp_df.item_id.groupby([temp_df.user_id, temp_df.item_id]).count().unstack().fillna(0).astype(int))
                sampled_df = pd.concat([sampled_df, temp_df])
            
            return sampled_df
        

        # Introduce sudden drift
        # No need to random sample, bc the list will have the frequencies for each item
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        # drift_items_list = [items_list[i] for i,x in enumerate(items_freq_list) if x == sudden_drift_start]
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}
        non_drift_items_list = list(set(items_list) - set(drift_items_list))
        
        print('drift_items_list', drift_items_list)
        print('renamed_items', renamed_items)
        print('non_drift_items_list', non_drift_items_list)

        
        random.seed(random_seed)  # For reproducibility
        sampled_df = sample_with_repetition_of_pattern(users_list,
                                                       non_drift_items_list,
                                                       non_drift_items_freq_list)
        
        sampled_df = pd.concat([sampled_df,
                                sample_with_repetition_of_pattern(users_list,
                                                                    drift_items_list,
                                                                    drift_items_freq_list)])

        if sampled_df.user_id.nunique() < n_users:
            # print(sampled_df.head())
            users_not_sampled = list(set(users_list) - set(sampled_df.user_id))
            print('users_not_sampled', len(users_not_sampled))
            # print('drift_items_list', drift_items_list)
            for user in users_not_sampled:
                for item in drift_items_list:
                    # print(sampled_df.loc[sampled_df['user_id']==user, 'item_id'].count())
                    sampled_df.loc[len(sampled_df)] = [user, item]


        sampled_df['item_id'] = sampled_df.apply(rename_item, axis=1)
        # print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))

        sampled_df['timestamp'] = ts



        sparsity, specs_str = calculate_sparsity(sampled_df)
        print('sparsity: ',sparsity)
        # print(specs_str)
        print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))
        # print(sampled_df.head())


        # when trainning on pt1, yield ValueError: Some users have interacted with all items, which we can not sample negative items for them. Please set `user_inter_num_interval` to filter those users.
        sampled_df = add_zero_user(sampled_df) # to solve the error 
        users_list.insert(0, 'u_0')

        # save_dataset_atomic_file(sampled_df, save_path, specs_str)
        # split_dataset_into_4_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        split_dataset_into_4_add_user0_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)

        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)


        return sampled_df

# Parameters
n_users = 4000 # bc of random sample, some users dont have occurrencies
n_items = 5
sudden_drift_start = 2000  # Starting user index for drift (1-indexed)
bin_size = 1000

string = "24/12/2024 21:12:24"
ts = time.mktime(datetime.strptime(string, "%d/%m/%Y %H:%M:%S").timetuple())

all_items_seen = False

random_seed = 42

ratio_to_drift = 2 # Select 50% of items to rename
n_items_to_drift = n_items // ratio_to_drift

base_filename = 'sudden_drift_dataset_u0_all_parts'
save_path = 'processed_datasets/artificial_data/'

drift_items_freq_list = [sudden_drift_start//2, sudden_drift_start//2]
non_drift_items_freq_list = [sudden_drift_start//4, sudden_drift_start//4, sudden_drift_start//4]

df = generate_artificial_random_dataset(n_users=n_users,
                                    n_items=n_items, 
                                    ts=ts,
                                    all_items_seen=all_items_seen,
                                    n_items_to_drift=n_items_to_drift,
                                    random_seed=random_seed,
                                    sudden_drift_start=sudden_drift_start,
                                    drift_items_freq_list=drift_items_freq_list,
                                    non_drift_items_freq_list=non_drift_items_freq_list,
                                    save_path=save_path,
                                    base_filename=base_filename,
                                    bin_size=bin_size)

drift_items_list ['i_1', 'i_5']
renamed_items {'i_1': 'drifted_i_1', 'i_5': 'drifted_i_5'}
non_drift_items_list ['i_2', 'i_3', 'i_4']
users_not_sampled 571
specs_str 4000x7_0.71
sparsity:  0.7092142857142858
item_id  drifted_i_1  drifted_i_5  i_1  i_2  i_3  i_4  i_5
user_id                                                   
u_1                0            0    1    0    0    0    0
u_10               0            0    1    0    0    0    1
u_100              0            0    0    0    0    0    1
u_1000             0            0    1    0    1    0    0
u_1001             0            0    1    1    1    0    1
...              ...          ...  ...  ...  ...  ...  ...
u_995              0            0    1    1    0    0    0
u_996              0            0    1    1    0    1    0
u_997              0            0    1    1    0    0    0
u_998              0            0    1    0    0    0    1
u_999              0            0    1    0    1    0    0

[4000 rows x 7 columns]


# re-do trigger error now with updated sampliing of df (properly separated items drift)

In [ ]:
# K = 3
# VALID_METRIC = 'Recall@'+str(K)
# MODEL = 'BPR'
# SEED = 2020
# USE_GPU = False
# SHUFFLE = False 
# SHOW_PROGRESS = False

## train test functions

In [10]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import BPR
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger


def train_test(model_name,
               dataset_name,
               parameter_dict):

    # configurations initialization
    config = Config(model=model_name, dataset=dataset_name, config_dict=parameter_dict)

    # init random seed
    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    model = BPR(config, train_data.dataset).to(config['device'])
    logger.info(model)

    # trainer loading and initialization
    trainer = Trainer(config, model)

    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    print('\n\nTraining best results')
    print('best_valid_score: ', best_valid_score)
    print('best_valid_result: ', best_valid_result)

    # model evaluation
    test_result = trainer.evaluate(test_data)
    
    print('\n\nTest results')
    print(test_result)

## Updated splittage version (user0 only in 1st part)

### train test Updated splittage version

In [74]:
base_filename = 'sudden_drift_dataset_updated'
save_path = 'processed_datasets/artificial_data/'


# Current model
base_dataset_name = base_filename+'_4000x7_0.71'
data_path = 'processed_datasets/artificial_data/'



In [17]:
dataset_name=base_dataset_name+'_pt1'
parameter_dict = {
    'dataset': dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':data_path+dataset_name,
    'seed':SEED,
    'shuffle': SHUFFLE
}


train_test(MODEL, dataset_name, parameter_dict)

09 Jan 11:28    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_updated_4000x7_0.71_pt1
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_updated_4000x7_0.71_pt1
show_progress =

 True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [3]
valid_metric = Recall@3
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = timestamp
seq_len = None
LABEL_FIELD = label
threshold = None
NEG_PR



Training best results
best_valid_score:  0.5556
best_valid_result:  OrderedDict([('recall@3', 0.5556), ('mrr@3', 0.3494), ('ndcg@3', 0.4018), ('hit@3', 0.5556), ('precision@3', 0.1852)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load



Test results
OrderedDict([('recall@3', 0.6332), ('mrr@3', 0.4412), ('ndcg@3', 0.4903), ('hit@3', 0.6332), ('precision@3', 0.2111)])


In [18]:
dataset_name=base_dataset_name+'_pt2'
parameter_dict = {
    'dataset': dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':data_path+dataset_name,
    'seed':SEED,
    'shuffle': SHUFFLE
}


train_test(MODEL, dataset_name, parameter_dict)

09 Jan 11:28    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_updated_4000x7_0.71_pt2
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_updated_4000x7_0.71_pt2
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'



Training best results
best_valid_score:  0.5253
best_valid_result:  OrderedDict([('recall@3', 0.5253), ('mrr@3', 0.3161), ('ndcg@3', 0.3694), ('hit@3', 0.5253), ('precision@3', 0.1751)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load



Test results
OrderedDict([('recall@3', 0.5726), ('mrr@3', 0.3619), ('ndcg@3', 0.4157), ('hit@3', 0.5726), ('precision@3', 0.1909)])


In [19]:
dataset_name=base_dataset_name+'_pt3'
parameter_dict = {
    'dataset': dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':data_path+dataset_name,
    'seed':SEED,
    'shuffle': SHUFFLE
}


train_test(MODEL, dataset_name, parameter_dict)

09 Jan 11:28    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_updated_4000x7_0.71_pt3
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_updated_4000x7_0.71_pt3
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'



Training best results
best_valid_score:  0.5122
best_valid_result:  OrderedDict([('recall@3', 0.5122), ('mrr@3', 0.3181), ('ndcg@3', 0.3676), ('hit@3', 0.5122), ('precision@3', 0.1707)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load



Test results
OrderedDict([('recall@3', 0.5517), ('mrr@3', 0.3444), ('ndcg@3', 0.3974), ('hit@3', 0.5517), ('precision@3', 0.1839)])


In [20]:
dataset_name=base_dataset_name+''
parameter_dict = {
    'dataset': dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':data_path+dataset_name,
    'seed':SEED,
    'shuffle': SHUFFLE
}


train_test(MODEL, dataset_name, parameter_dict)

09 Jan 11:28    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_updated_4000x7_0.71
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_updated_4000x7_0.71
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk =



Training best results
best_valid_score:  0.5455
best_valid_result:  OrderedDict([('recall@3', 0.5455), ('mrr@3', 0.3223), ('ndcg@3', 0.379), ('hit@3', 0.5455), ('precision@3', 0.1818)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load



Test results
OrderedDict([('recall@3', 0.5476), ('mrr@3', 0.3279), ('ndcg@3', 0.3838), ('hit@3', 0.5476), ('precision@3', 0.1825)])


### trigger error in Updated splittage version

#### trigger_error(model_ver='_pt1', data_ver='_pt2') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_11-28-25', data_ver='_pt2')

#### trigger_error(model_ver='_pt1', data_ver='_pt3') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_11-28-25', data_ver='_pt3')

#### trigger_error(model_ver='_pt1', data_ver='_pt5') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_11-28-25', data_ver='_pt5')

#### trigger_error(model_ver='_pt1', data_ver='_pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_11-28-25', data_ver='_pt6')

#### trigger_error(model_ver='_pt1', data_ver='_pt7') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_11-28-25', data_ver='_pt7')

#### trigger_error(model_ver='_pt1', data_ver='_pt8') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_11-28-25', data_ver='_pt8')

#### trigger_error(model_ver='_pt1', data_ver='') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_11-28-25', data_ver='')

#### trigger_error(model_ver='_pt2', data_ver='_pt1') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_11-28-37', data_ver='_pt1')

#### trigger_error(model_ver='_pt2', data_ver='_pt3') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_11-28-37', data_ver='_pt3')

#### trigger_error(model_ver='_pt2', data_ver='_pt5') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_11-28-37', data_ver='_pt5')

#### trigger_error(model_ver='_pt2', data_ver='_pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_11-28-37', data_ver='_pt6')

#### trigger_error(model_ver='_pt2', data_ver='_pt7') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_11-28-37', data_ver='_pt7')

#### trigger_error(model_ver='_pt2', data_ver='_pt8') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_11-28-37', data_ver='_pt8')

#### trigger_error(model_ver='_pt2', data_ver='') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_11-28-37', data_ver='')

#### trigger_error(model_ver='_pt3', data_ver='_pt1') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_11-28-42', data_ver='_pt1')

#### trigger_error(model_ver='_pt3', data_ver='_pt2') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_11-28-42', data_ver='_pt2')

#### trigger_error(model_ver='_pt3', data_ver='pt5') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_11-28-42', data_ver='_pt5')

#### trigger_error(model_ver='_pt3', data_ver='pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_11-28-42', data_ver='_pt6')

#### trigger_error(model_ver='_pt3', data_ver='_pt7') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_11-28-42', data_ver='_pt7')

#### trigger_error(model_ver='_pt3', data_ver='_pt8') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_11-28-42', data_ver='_pt8')

#### trigger_error(model_ver='_pt3', data_ver='') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_11-28-42', data_ver='')

#### trigger_error(model_ver='', data_ver='_pt1') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_11-28-47', data_ver='_pt1')

#### trigger_error(model_ver='', data_ver='_pt2') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_11-28-47', data_ver='_pt2')

#### trigger_error(model_ver='', data_ver='_pt3') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_11-28-47', data_ver='_pt3')

#### trigger_error(model_ver='', data_ver='_pt5') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_11-28-47', data_ver='_pt5')

#### trigger_error(model_ver='', data_ver='_pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_11-28-47', data_ver='_pt6')

#### trigger_error(model_ver='', data_ver='_pt7') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_11-28-47', data_ver='_pt7')

#### trigger_error(model_ver='', data_ver='_pt8') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_11-28-47', data_ver='_pt8')

## Updated (user0 ALL part) splittage version 

In [94]:
base_filename = 'sudden_drift_dataset_u0_all_parts'
save_path = 'processed_datasets/artificial_data/'


# Current model
base_dataset_name = base_filename+'_4000x7_0.71'
data_path = 'processed_datasets/artificial_data/'



### train test Updated user0 all parts version

In [42]:
dataset_name=base_dataset_name+'_pt1'
parameter_dict = {
    'dataset': dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':data_path+dataset_name,
    'seed':SEED,
    'shuffle': SHUFFLE
}


train_test(MODEL, dataset_name, parameter_dict)

09 Jan 16:25    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_u0_all_parts_4000x7_0.71_pt1
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_u0_all_parts_4000x7_0.71_pt1
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', '



Training best results
best_valid_score:  0.5609
best_valid_result:  OrderedDict([('recall@3', 0.5609), ('mrr@3', 0.3493), ('ndcg@3', 0.4034), ('hit@3', 0.5609), ('precision@3', 0.187)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load



Test results
OrderedDict([('recall@3', 0.5551), ('mrr@3', 0.3361), ('ndcg@3', 0.392), ('hit@3', 0.5551), ('precision@3', 0.185)])


In [43]:
dataset_name=base_dataset_name+'_pt2'
parameter_dict = {
    'dataset': dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':data_path+dataset_name,
    'seed':SEED,
    'shuffle': SHUFFLE
}


train_test(MODEL, dataset_name, parameter_dict)

09 Jan 16:25    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_u0_all_parts_4000x7_0.71_pt2
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_u0_all_parts_4000x7_0.71_pt2
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', '



Training best results
best_valid_score:  0.5573
best_valid_result:  OrderedDict([('recall@3', 0.5573), ('mrr@3', 0.3482), ('ndcg@3', 0.4015), ('hit@3', 0.5573), ('precision@3', 0.1858)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load



Test results
OrderedDict([('recall@3', 0.5663), ('mrr@3', 0.3361), ('ndcg@3', 0.3947), ('hit@3', 0.5663), ('precision@3', 0.1888)])


In [44]:
dataset_name=base_dataset_name+'_pt3'
parameter_dict = {
    'dataset': dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':data_path+dataset_name,
    'seed':SEED,
    'shuffle': SHUFFLE
}


train_test(MODEL, dataset_name, parameter_dict)

09 Jan 16:25    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_u0_all_parts_4000x7_0.71_pt3
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_u0_all_parts_4000x7_0.71_pt3
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', '



Training best results
best_valid_score:  0.5423
best_valid_result:  OrderedDict([('recall@3', 0.5423), ('mrr@3', 0.325), ('ndcg@3', 0.3803), ('hit@3', 0.5423), ('precision@3', 0.1808)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load



Test results
OrderedDict([('recall@3', 0.5602), ('mrr@3', 0.3478), ('ndcg@3', 0.4019), ('hit@3', 0.5602), ('precision@3', 0.1867)])


In [45]:
dataset_name=base_dataset_name+''
parameter_dict = {
    'dataset': dataset_name+'.inter',
    'data_path': data_path,
    'load_col': {'inter': ['user_id', 'item_id']},
    'use_gpu':USE_GPU,
    'topk':K,
    'valid_metric':VALID_METRIC,
    'checkpoint_dir':data_path+dataset_name,
    'seed':SEED,
    'shuffle': SHUFFLE
}


train_test(MODEL, dataset_name, parameter_dict)

09 Jan 16:25    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_u0_all_parts_4000x7_0.71
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_u0_all_parts_4000x7_0.71
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precisio



Training best results
best_valid_score:  0.6016
best_valid_result:  OrderedDict([('recall@3', 0.6016), ('mrr@3', 0.3836), ('ndcg@3', 0.4392), ('hit@3', 0.6016), ('precision@3', 0.2005)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load



Test results
OrderedDict([('recall@3', 0.6734), ('mrr@3', 0.4335), ('ndcg@3', 0.4948), ('hit@3', 0.6734), ('precision@3', 0.2245)])


### trigger error in Updated (user0 ALL parts) splittage version

#### trigger_error(model_ver='_pt1', data_ver='pt_2') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_16-25-11', data_ver='_pt2')

#### trigger_error(model_ver='_pt1', data_ver='pt_3') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_16-25-11', data_ver='_pt3')

#### trigger_error(model_ver='_pt1', data_ver='_pt5') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_16-25-11', data_ver='_pt5')

#### trigger_error(model_ver='_pt1', data_ver='_pt6') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_16-25-11', data_ver='_pt6')

#### trigger_error(model_ver='_pt1', data_ver='_pt7') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_16-25-11', data_ver='_pt7')

#### trigger_error(model_ver='_pt1', data_ver='_pt8') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_16-25-11', data_ver='_pt8')

#### trigger_error(model_ver='_pt1', data_ver='') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-09-2025_16-25-11', data_ver='')

#### trigger_error(model_ver='_pt2', data_ver='_pt1') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_16-25-14', data_ver='_pt1')

#### trigger_error(model_ver='_pt2', data_ver='_pt3') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_16-25-14', data_ver='_pt3')

#### trigger_error(model_ver='_pt2', data_ver='_pt5') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_16-25-14', data_ver='_pt5')

#### trigger_error(model_ver='_pt2', data_ver='_pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_16-25-14', data_ver='_pt6')

#### trigger_error(model_ver='_pt2', data_ver='_pt7') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_16-25-14', data_ver='_pt7')

#### trigger_error(model_ver='_pt2', data_ver='_pt8') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_16-25-14', data_ver='_pt8')

#### trigger_error(model_ver='_pt2', data_ver='') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-09-2025_16-25-14', data_ver='')

#### trigger_error(model_ver='_pt3', data_ver='_pt1') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_16-25-16', data_ver='_pt1')

#### trigger_error(model_ver='_pt3', data_ver='_pt2') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_16-25-16', data_ver='_pt2')

#### trigger_error(model_ver='_pt3', data_ver='_pt5') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_16-25-16', data_ver='_pt5')

#### trigger_error(model_ver='_pt3', data_ver='_pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_16-25-16', data_ver='_pt6')

#### trigger_error(model_ver='_pt3', data_ver='_pt7') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_16-25-16', data_ver='_pt7')

#### trigger_error(model_ver='_pt3', data_ver='_pt8') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_16-25-16', data_ver='_pt8')

#### trigger_error(model_ver='_pt3', data_ver='') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-09-2025_16-25-16', data_ver='')

#### trigger_error(model_ver='', data_ver='_pt1') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_16-25-19', data_ver='_pt1')

#### trigger_error(model_ver='', data_ver='_pt2') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_16-25-19', data_ver='_pt2')

#### trigger_error(model_ver='', data_ver='_pt3') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_16-25-19', data_ver='_pt3')

#### trigger_error(model_ver='', data_ver='_pt5') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_16-25-19', data_ver='_pt5')

#### trigger_error(model_ver='', data_ver='_pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_16-25-19', data_ver='_pt6')

#### trigger_error(model_ver='', data_ver='_pt7') (UsI e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_16-25-19', data_ver='_pt7')

#### trigger_error(model_ver='', data_ver='_pt8') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-09-2025_16-25-19', data_ver='_pt8')

# re-generate data with added user0 seeing item 1 and 5

In [12]:
import os
import pandas as pd
import random
from datetime import datetime
import time
import pickle


def validate_folderpath(folderpath):
    # print('im validating')
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)
        print('Folder created: ', folderpath)


def save_picklefile(d, filepath):
    with open(filepath, 'wb') as f:
            pickle.dump(d, f)

    print('Saved file at '+filepath)

def load_picklefile(filepath):
    with open(filepath, 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

    


def generate_artificial_random_dataset(n_users,
                                       n_items, 
                                       ts,
                                       all_items_seen,
                                       random_seed,
                                       n_items_to_drift,
                                       sudden_drift_start,
                                       drift_items_freq_list,
                                       non_drift_items_freq_list,
                                       save_path,
                                       base_filename,
                                       bin_size):
    
    def add_zero_user(df):

        df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
        df['user_id_n'] = df['user_id_n'].astype(int)

        df.loc[-1] = ['u_0', 'i_1', ts, 0]
        df.loc[-4] = ['u_0', 'i_5', ts, 0]
        # df.loc[-2] = ['u_-1', 'i_2', ts, 0]
        # df.loc[-3] = ['u_-1', 'i_3', ts, 0]
        # df.loc[-5] = ['u_0', 'i_4', ts, 0]
        df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
        df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

        df.sort_values(by='user_id_n', inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.drop(columns=['user_id_n'], inplace=True)

        # print('added zero user\n', df.head())
        return df

    
    def create_folderpath(save_path, base_filename, specs_str):
        return save_path+base_filename+'_'+specs_str+'/'
    

    def save_items_frequencies(n_items_to_drift,
                               sudden_drift_start,
                               drift_items_freq_list,
                               non_drift_items_freq_list,
                               save_path, 
                               base_filename,
                               specs_str):
        
        folderpath = create_folderpath(save_path, base_filename, specs_str)
        validate_folderpath(folderpath)

        d = {'n_items_to_drift': n_items_to_drift,
             'sudden_drift_start': sudden_drift_start,
             'drift_items_freq_list': drift_items_freq_list,
             'non_drift_items_freq_list': non_drift_items_freq_list}

        save_picklefile(d, folderpath+'saved_dictionary.pkl')
       

    
    def save_dataset_atomic_file(df, user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())


    def add_user0_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)
            df_sampled = add_zero_user(df_sampled)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())




    def split_dataset_into_4_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

        
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def split_dataset_into_4_add_user0_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        add_user0_save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

                
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def calculate_sparsity(df):
        # df.item_id.groupby([df.user_id, df.item_id]).count().sum() == df.user_id.count()
        sparsity = 1 - df.user_id.count()/(df.user_id.nunique()*df.item_id.nunique())
        specs_str = str(df.user_id.nunique())+'x'+str(df.item_id.nunique())+'_'+str(round(sparsity, 2))
        print('specs_str', specs_str)
        return sparsity, specs_str
    

    def rename_item(row):
        if int(row['user_id'].split('_')[1]) > sudden_drift_start and row['item_id'] in renamed_items:
            return renamed_items[row['item_id']]
        return row['item_id']




    # users_list = [f'u_{i+1}' for i in range(1, n_users)]
    users_list = [f'u_{i+1}' for i in range(n_users)]
    items_list = [f'i_{j+1}' for j in range(n_items)]


    if all_items_seen:

        data = []
        for user in users_list:
            for item in items_list:
                data.append({'user_id': user, 'item_id': item, 'timestamp':ts})

        all_items_seen_df = pd.DataFrame(data)

        # Introduce sudden drift
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}       
        

        all_items_seen_df['item_id'] = all_items_seen_df.apply(rename_item, axis=1)
        # print(all_items_seen_df.item_id.groupby([all_items_seen_df.user_id, all_items_seen_df.item_id]).count().unstack().fillna(0).astype(int))

        
        sparsity , specs_str = calculate_sparsity(all_items_seen_df)
        print('sparsity: ',sparsity)

        sampled_df = add_zero_user(sampled_df)
        users_list.insert(0, 'u_0')
        

        # split_dataset_into_4_and_save_atomic_file(all_items_seen_df,users_list, bin_size, save_path, specs_str)
        # save_dataset_atomic_file(all_items_seen_df, save_path, specs_str)
        split_dataset_into_4_add_user0_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)
        

    else:
        
        if len(drift_items_freq_list) != n_items_to_drift:
            print('Not all items frequency was specified!')
            return None
        elif len(non_drift_items_freq_list) != len(items_list)-n_items_to_drift:
            print('Not all items frequency was specified!')
            return None


        def sample_with_repetition_of_pattern(users_list, items_list, items_freq_list):
            random.seed(random_seed)
            sampled_df = pd.DataFrame({})
            for i, freq in enumerate(items_freq_list):
                # print('k ',k)
                user_sample = random.sample(users_list[:sudden_drift_start], k=freq) +\
                                random.sample(users_list[sudden_drift_start:], k=freq)
                temp_df = pd.DataFrame({'user_id': user_sample, 
                                        'item_id': items_list[i]})
                # print(temp_df.item_id.groupby([temp_df.user_id, temp_df.item_id]).count().unstack().fillna(0).astype(int))
                sampled_df = pd.concat([sampled_df, temp_df])
            
            return sampled_df
        

        # Introduce sudden drift
        # No need to random sample, bc the list will have the frequencies for each item
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        # drift_items_list = [items_list[i] for i,x in enumerate(items_freq_list) if x == sudden_drift_start]
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}
        non_drift_items_list = list(set(items_list) - set(drift_items_list))
        
        print('drift_items_list', drift_items_list)
        print('renamed_items', renamed_items)
        print('non_drift_items_list', non_drift_items_list)

        
        random.seed(random_seed)  # For reproducibility
        sampled_df = sample_with_repetition_of_pattern(users_list,
                                                       non_drift_items_list,
                                                       non_drift_items_freq_list)
        
        sampled_df = pd.concat([sampled_df,
                                sample_with_repetition_of_pattern(users_list,
                                                                    drift_items_list,
                                                                    drift_items_freq_list)])

        if sampled_df.user_id.nunique() < n_users:
            # print(sampled_df.head())
            users_not_sampled = list(set(users_list) - set(sampled_df.user_id))
            print('users_not_sampled', len(users_not_sampled))
            # print('drift_items_list', drift_items_list)
            for user in users_not_sampled:
                for item in drift_items_list:
                    # print(sampled_df.loc[sampled_df['user_id']==user, 'item_id'].count())
                    sampled_df.loc[len(sampled_df)] = [user, item]


        sampled_df['item_id'] = sampled_df.apply(rename_item, axis=1)
        # print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))

        sampled_df['timestamp'] = ts



        sparsity, specs_str = calculate_sparsity(sampled_df)
        print('sparsity: ',sparsity)
        # print(specs_str)
        print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))
        # print(sampled_df.head())


        # when trainning on pt1, yield ValueError: Some users have interacted with all items, which we can not sample negative items for them. Please set `user_inter_num_interval` to filter those users.
        # sampled_df = add_zero_user(sampled_df) # to solve the error 
        users_list.insert(0, 'u_0')

        # save_dataset_atomic_file(sampled_df, save_path, specs_str)
        # split_dataset_into_4_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        split_dataset_into_4_add_user0_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)

        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)


        return sampled_df

# Parameters
n_users = 4000 # bc of random sample, some users dont have occurrencies
n_items = 5
sudden_drift_start = 2000  # Starting user index for drift (1-indexed)
bin_size = 1000

string = "24/12/2024 21:12:24"
ts = time.mktime(datetime.strptime(string, "%d/%m/%Y %H:%M:%S").timetuple())

all_items_seen = False

random_seed = 42

ratio_to_drift = 2 # Select 50% of items to rename
n_items_to_drift = n_items // ratio_to_drift

base_filename = 'sudden_drift_dataset_u0_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

drift_items_freq_list = [sudden_drift_start//2, sudden_drift_start//2]
non_drift_items_freq_list = [sudden_drift_start//4, sudden_drift_start//4, sudden_drift_start//4]

df = generate_artificial_random_dataset(n_users=n_users,
                                    n_items=n_items, 
                                    ts=ts,
                                    all_items_seen=all_items_seen,
                                    n_items_to_drift=n_items_to_drift,
                                    random_seed=random_seed,
                                    sudden_drift_start=sudden_drift_start,
                                    drift_items_freq_list=drift_items_freq_list,
                                    non_drift_items_freq_list=non_drift_items_freq_list,
                                    save_path=save_path,
                                    base_filename=base_filename,
                                    bin_size=bin_size)

drift_items_list ['i_1', 'i_5']
renamed_items {'i_1': 'drifted_i_1', 'i_5': 'drifted_i_5'}
non_drift_items_list ['i_4', 'i_3', 'i_2']
users_not_sampled 566
specs_str 3999x7_0.71
sparsity:  0.7094988032722467
item_id  drifted_i_1  drifted_i_5  i_1  i_2  i_3  i_4  i_5
user_id                                                   
u_10               0            0    1    0    0    0    1
u_100              0            0    1    0    0    0    1
u_1000             0            0    1    0    1    0    0
u_1001             0            0    1    0    1    0    0
u_1002             0            0    1    0    1    1    1
...              ...          ...  ...  ...  ...  ...  ...
u_995              0            0    1    0    0    0    1
u_996              0            0    1    0    0    1    0
u_997              0            0    1    1    0    1    0
u_998              0            0    1    0    0    1    0
u_999              0            0    1    0    0    0    1

[3999 rows x 7 columns]


okay, now all parts have all items

## train test functions

In [9]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import BPR
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger


def train_test(model_name,
               dataset_name,
               parameter_dict):

    # configurations initialization
    config = Config(model=model_name, dataset=dataset_name, config_dict=parameter_dict)

    # init random seed
    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    model = BPR(config, train_data.dataset).to(config['device'])
    logger.info(model)

    # trainer loading and initialization
    trainer = Trainer(config, model)

    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    print('\n\nTraining best results')
    print('best_valid_score: ', best_valid_score)
    print('best_valid_result: ', best_valid_result)

    # model evaluation
    test_result = trainer.evaluate(test_data)
    
    print('\n\nTest results')
    print(test_result)

## train test Updated user0 sees i1 i5 i1_drift i5_drift in ALL parts splittage version

In [7]:
base_filename = 'sudden_drift_dataset_u0_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

# Current model~~~~
base_dataset_name = base_filename+'_4000x7_0.71'
data_path = 'processed_datasets/artificial_data/'

In [15]:
for part in ['_pt1', '_pt2', '_pt3', '']:
    print('\n\n'+part)
    dataset_name=base_dataset_name+part
    parameter_dict = {
        'dataset': dataset_name+'.inter',
        'data_path': data_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        'valid_metric':VALID_METRIC,
        'checkpoint_dir':data_path+dataset_name,
        'seed':SEED,
        'shuffle': SHUFFLE
    }


    train_test(MODEL, dataset_name, parameter_dict)



_pt1


12 Jan 16:33    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_u0_i1i5_drift_all_parts_4000x7_0.71_pt1
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_u0_i1i5_drift_all_parts_4000x7_0.71_pt1
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', '

User IDs: tensor([648, 648, 648,  ..., 223, 847, 807])
User ID Range: 1 to 1000
Embedding Weight Shape: torch.Size([1001, 64])


12 Jan 16:33    INFO  epoch 0 training [time: 0.07s, train loss: 0.6935]


User IDs: tensor([  1,   5,   8,   9,  15,  18,  21,  25,  30,  43,  56,  66,  67,  68,
         77,  83,  84,  86,  87,  91,  92,  95,  98, 105, 106, 107, 111, 116,
        118, 119, 120, 125, 126, 132, 140, 141, 142, 143, 144, 147, 148, 152,
        153, 155, 156, 163, 165, 166, 168, 169, 175, 177, 188, 191, 193, 195,
        200, 203, 206, 207, 208, 212, 214, 219, 221, 225, 226, 231, 235, 244,
        249, 250, 256, 257, 260, 264, 269, 270, 272, 275, 285, 289, 294, 304,
        307, 312, 315, 317, 319, 324, 325, 328, 329, 334, 339, 342, 352, 357,
        360, 367, 374, 390, 391, 393, 394, 395, 396, 399, 409, 410, 413, 418,
        420, 433, 434, 435, 440, 442, 449, 455, 461, 462, 464, 468, 470, 473,
        478, 486, 490, 491, 492, 494, 495, 499, 500, 502, 503, 506, 509, 510,
        513, 516, 518, 536, 538, 543, 544, 545, 548, 559, 560, 561, 564, 565,
        568, 571, 572, 574, 579, 581, 583, 584, 585, 586, 587, 589, 592, 594,
        598, 602, 603, 607, 613, 615, 616, 621, 624, 6

...

User IDs: tensor([   1,    5,    8,    9,   15,   18,   21,   25,   30,   43,   56,   66,
          67,   68,   77,   83,   84,   86,   87,   91,   92,   95,   98,  105,
         106,  107,  111,  116,  118,  119,  120,  125,  126,  132,  140,  141,
         142,  143,  144,  147,  148,  152,  153,  155,  156,  163,  165,  166,
         168,  169,  175,  177,  188,  191,  193,  195,  200,  203,  206,  207,
         208,  212,  214,  219,  221,  225,  226,  231,  235,  244,  249,  250,
         256,  257,  260,  264,  269,  270,  272,  275,  285,  289,  294,  304,
         307,  312,  315,  317,  319,  324,  325,  328,  329,  334,  339,  342,
         352,  357,  360,  367,  374,  390,  391,  393,  394,  395,  396,  399,
         409,  410,  413,  418,  420,  433,  434,  435,  440,  442,  449,  455,
         461,  462,  464,  468,  470,  473,  478,  486,  490,  491,  492,  494,
         495,  499,  500,  502,  503,  506,  509,  510,  513,  516,  518,  536,
         538,  543,  544,  545

12 Jan 16:34    INFO  epoch 11 evaluating [time: 0.29s, valid_score: 0.486300]
12 Jan 16:34    INFO  valid result: 
recall@3 : 0.4863    mrr@3 : 0.2957    ndcg@3 : 0.3443    hit@3 : 0.4863    precision@3 : 0.1621
12 Jan 16:34    INFO  Finished training, best eval result in epoch 0




Training best results
best_valid_score:  0.5146
best_valid_result:  OrderedDict([('recall@3', 0.5146), ('mrr@3', 0.3123), ('ndcg@3', 0.3639), ('hit@3', 0.5146), ('precision@3', 0.1715)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load

User IDs: tensor([  1,   3,   5,   6,   7,   8,   9,  10,  11,  12,  15,  16,  17,  18,
         19,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
         35,  37,  38,  40,  41,  43,  44,  45,  47,  51,  53,  54,  56,  60,
         61,  63,  65,  66,  67,  68,  69,  71,  72,  75,  76,  77,  80,  81,
         82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  95,  96,
         97,  98,  99, 100, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127,
        128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 150, 151, 152, 153, 155, 156, 159, 160, 161,
        162, 163, 165, 166, 168, 169, 170, 172, 173, 174, 175, 176, 177, 179,
        180, 182, 183, 184, 186, 188, 189, 191, 192, 193, 194, 195, 196, 198,
        199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 211, 212, 214, 215,
        217, 218, 219, 221, 225, 226, 228, 229, 230, 2

## trigger_error

In [10]:
def test_on_earlier_data(model_name,
                         earlier_datasets,
                         current_dataset_name,
                         parameter_dict,
                         checkpoint_file):


    current_config,\
        current_logger,\
                current_dataset,\
                    current_train_data, current_valid_data, current_test_data = setup_config_and_dataset(model_name,
                                                                                current_dataset_name,
                                                                                parameter_dict)

    # model loading and initialization
    current_model = BPR(current_config, current_train_data.dataset).to(current_config['device'])
    current_logger.info(current_model)


    # trainer loading and initialization
    trainer = get_trainer(current_config['MODEL_TYPE'], current_config['model'])(current_config, current_model)


    # results = []

    for earlier_dataset_name in earlier_datasets:
        print('\n\n'+earlier_dataset_name)
        earlier_config,\
        earlier_logger,\
                earlier_dataset,\
                    earlier_train_data, earlier_valid_data, earlier_test_data = setup_config_and_dataset(model_name,
                                                                                earlier_dataset_name,
                                                                                parameter_dict)


        # When calculate ItemCoverage metrics, we need to run this code for set item_nums in eval_collector.
        # trainer.eval_collector.data_collect(earlier_train_data)
        trainer.eval_collector.data_collect(current_train_data)

        # model evaluation
        test_result = trainer.evaluate(earlier_test_data, model_file=checkpoint_file)
        # results += [test_result]
    
        print(test_result)


def trigger_error(data_path, base_dataset_name, model_ver, checkpoint_ver, data_ver):
    current_ver = model_ver
    current_dataset_name = base_dataset_name+current_ver

    earlier_datasets = [base_dataset_name+data_ver]#,base_dataset_name+'_pt6', base_dataset_name+'_pt7', base_dataset_name+'_pt8'] # 1,5,6,7,8


    # Checkpoint - 
    # checkpoint_ver = 'BPR-Jan-01-2025_16-20-32'
    checkpoint_dir = data_path+base_dataset_name+current_ver
    checkpoint_file = checkpoint_dir+'/'+checkpoint_ver+'.pth'


    parameter_dict = {
        'dataset': current_dataset_name+'.inter',
        'data_path': data_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        'valid_metric':VALID_METRIC,
        # 'user_inter_num_interval':'[1,inf)',
        'checkpoint_dir':checkpoint_dir,
        'seed':SEED,
        'shuffle': SHUFFLE,
        'state':'ERROR',
        # 'show_progress': SHOW_PROGRESS,
        # https://recbole.io/docs/user_guide/config/evaluation_settings.html
        'eval_args': {'split': {'LS': 'test_only'}, # leave-one-out sample type
                    'group_by': 'user',
                    'order': 'RO', # random ordering
                    'mode': 'pop001'} #  for example pop100, means sample 100 negative items for each positive item in testing set based on item popularity (Counter(item) in .inter file), and evaluate the model on these positive items with their sampled negative items
    }

    test_on_earlier_data(MODEL,
                    earlier_datasets,
                    current_dataset_name,
                    parameter_dict, 
                    checkpoint_file)

I expect all data_ver='_pt7' to yield Index ou of Range error

### trigger_error(model_ver='_pt2', data_ver='_pt1') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-10-2025_12-00-43', data_ver='_pt1')

### trigger_error(model_ver='_pt3', data_ver='_pt1') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-10-2025_12-00-48', data_ver='_pt1')

### trigger_error(model_ver='', data_ver='_pt1') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-10-2025_12-00-55', data_ver='_pt1')

### trigger_error(model_ver='_pt1', data_ver='_pt2') (IoR)

In [13]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_14-26-30', data_ver='_pt2')

c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\data\dataset\dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)




sudden_drift_dataset_u0_i1i5_drift_all_parts_4000x7_0.71_pt2


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\data\dataset\dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code d

User ID Range: 1 to 2000
Embedding Weight Shape: torch.Size([1001, 64])


IndexError: index out of range in self

### trigger_error(model_ver='_pt3', data_ver='_pt2') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-10-2025_12-00-48', data_ver='_pt2')

### trigger_error(model_ver='', data_ver='_pt2') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-10-2025_12-00-55', data_ver='_pt2')

### trigger_error(model_ver='_pt1', data_ver='_pt3') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_14-26-30', data_ver='_pt3')

### trigger_error(model_ver='_pt2', data_ver='_pt3') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-10-2025_12-00-43', data_ver='_pt3')

### trigger_error(model_ver='', data_ver='_pt3') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-10-2025_12-00-55', data_ver='_pt3')

### trigger_error(model_ver='_pt1', data_ver='') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_14-26-30', data_ver='')

### trigger_error(model_ver='_pt2', data_ver='') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-10-2025_12-00-43', data_ver='')

### trigger_error(model_ver='_pt3', data_ver='') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-10-2025_12-00-48', data_ver='')

### trigger_error(model_ver='_pt1', data_ver='_pt6') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_14-26-30', data_ver='_pt6')

### trigger_error(model_ver='_pt2', data_ver='_pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-10-2025_12-00-43', data_ver='_pt6')

### trigger_error(model_ver='_pt3', data_ver='_pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-10-2025_12-00-48', data_ver='_pt6')

### trigger_error(model_ver='', data_ver='_pt6') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-10-2025_12-00-55', data_ver='_pt6')

### trigger_error(model_ver='_pt1', data_ver='_pt8') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_14-26-30', data_ver='_pt8')

### trigger_error(model_ver='_pt2', data_ver='_pt8') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-10-2025_12-00-43', data_ver='_pt8')

### trigger_error(model_ver='_pt3', data_ver='_pt8') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-10-2025_12-00-48', data_ver='_pt8')

### trigger_error(model_ver='', data_ver='_pt8') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-10-2025_12-00-55', data_ver='_pt8')

### trigger_error(model_ver='_pt1', data_ver='_pt5') (IoR e)

In [17]:
# trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_12-00-41', data_ver='_pt5')
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_14-26-30', data_ver='_pt5')

12 Jan 16:34    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = ERROR
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_u0_i1i5_drift_all_parts_4000x7_0.71_pt1
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_u0_i1i5_drift_all_parts_4000x7_0.71_pt1
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'test_only'}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'pop001', 'test': 'pop001'}}
repeatable = False
metrics = ['Recall', 



sudden_drift_dataset_u0_i1i5_drift_all_parts_4000x7_0.71_pt5


12 Jan 16:34    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = ERROR
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_u0_i1i5_drift_all_parts_4000x7_0.71_pt5
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_u0_i1i5_drift_all_parts_4000x7_0.71_pt1
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'test_only'}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'pop001', 'test': 'pop001'}}
repeatable = False
metrics = ['Recall', 

User IDs: tensor([   1,    1,    2,  ...,  997, 1001, 1001])
User ID Range: 1 to 1001
Embedding Weight Shape: torch.Size([1001, 64])


IndexError: index out of range in self

### trigger_error(model_ver='_pt2', data_ver='_pt5') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-10-2025_12-00-43', data_ver='_pt5')

### trigger_error(model_ver='_pt3', data_ver='_pt5') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-10-2025_12-00-48', data_ver='_pt5')

### trigger_error(model_ver='', data_ver='_pt5') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-10-2025_12-00-55', data_ver='_pt5')

### trigger_error(model_ver='_pt1', data_ver='_pt7') (IoR e)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_12-00-41', data_ver='_pt7')

### trigger_error(model_ver='_pt2', data_ver='_pt7') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-10-2025_12-00-43', data_ver='_pt7')

### trigger_error(model_ver='_pt3', data_ver='_pt7') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt3', checkpoint_ver = 'BPR-Jan-10-2025_12-00-48', data_ver='_pt7')

### trigger_error(model_ver='', data_ver='_pt7') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='', checkpoint_ver = 'BPR-Jan-10-2025_12-00-55', data_ver='_pt7')

# change what data is fed to eval_colector re-doing some trigger error

In [18]:
def test_on_earlier_data(model_name,
                         earlier_datasets,
                         current_dataset_name,
                         parameter_dict,
                         checkpoint_file):


    current_config,\
        current_logger,\
                current_dataset,\
                    current_train_data, current_valid_data, current_test_data = setup_config_and_dataset(model_name,
                                                                                current_dataset_name,
                                                                                parameter_dict)

    # model loading and initialization
    current_model = BPR(current_config, current_train_data.dataset).to(current_config['device'])
    current_logger.info(current_model)


    # trainer loading and initialization
    trainer = get_trainer(current_config['MODEL_TYPE'], current_config['model'])(current_config, current_model)


    # results = []

    for earlier_dataset_name in earlier_datasets:
        print('\n\n'+earlier_dataset_name)
        earlier_config,\
        earlier_logger,\
                earlier_dataset,\
                    earlier_train_data, earlier_valid_data, earlier_test_data = setup_config_and_dataset(model_name,
                                                                                earlier_dataset_name,
                                                                                parameter_dict)


        # When calculate ItemCoverage metrics, we need to run this code for set item_nums in eval_collector.
        trainer.eval_collector.data_collect(earlier_train_data)
        # trainer.eval_collector.data_collect(current_train_data)

        # model evaluation
        test_result = trainer.evaluate(earlier_test_data, model_file=checkpoint_file)
        # results += [test_result]
    
        print(test_result)


def trigger_error(data_path, base_dataset_name, model_ver, checkpoint_ver, data_ver):
    current_ver = model_ver
    current_dataset_name = base_dataset_name+current_ver

    earlier_datasets = [base_dataset_name+data_ver]#,base_dataset_name+'_pt6', base_dataset_name+'_pt7', base_dataset_name+'_pt8'] # 1,5,6,7,8


    # Checkpoint - 
    # checkpoint_ver = 'BPR-Jan-01-2025_16-20-32'
    checkpoint_dir = data_path+base_dataset_name+current_ver
    checkpoint_file = checkpoint_dir+'/'+checkpoint_ver+'.pth'


    parameter_dict = {
        'dataset': current_dataset_name+'.inter',
        'data_path': data_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        'valid_metric':VALID_METRIC,
        # 'user_inter_num_interval':'[1,inf)',
        'checkpoint_dir':checkpoint_dir,
        'seed':SEED,
        'shuffle': SHUFFLE,
        'state':'ERROR',
        # 'show_progress': SHOW_PROGRESS,
        # https://recbole.io/docs/user_guide/config/evaluation_settings.html
        'eval_args': {'split': {'LS': 'test_only'}, # leave-one-out sample type
                    'group_by': 'user',
                    'order': 'RO', # random ordering
                    'mode': 'pop001'} #  for example pop100, means sample 100 negative items for each positive item in testing set based on item popularity (Counter(item) in .inter file), and evaluate the model on these positive items with their sampled negative items
    }

    test_on_earlier_data(MODEL,
                    earlier_datasets,
                    current_dataset_name,
                    parameter_dict, 
                    checkpoint_file)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_12-00-41', data_ver='_pt2')

### trigger_error(model_ver='_pt1', data_ver='_pt7') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_12-00-41', data_ver='_pt7')

### trigger_error(model_ver='_pt2', data_ver='_pt7') (pass)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-10-2025_12-00-43', data_ver='_pt7')

# re-generate data with new user indexes

In [ ]:
import os
import pandas as pd
import random
from datetime import datetime
import time
import pickle


def validate_folderpath(folderpath):
    # print('im validating')
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)
        print('Folder created: ', folderpath)


def save_picklefile(d, filepath):
    with open(filepath, 'wb') as f:
            pickle.dump(d, f)

    print('Saved file at '+filepath)

def load_picklefile(filepath):
    with open(filepath, 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

    


def generate_artificial_random_dataset(n_users,
                                       n_items, 
                                       ts,
                                       all_items_seen,
                                       random_seed,
                                       n_items_to_drift,
                                       sudden_drift_start,
                                       drift_items_freq_list,
                                       non_drift_items_freq_list,
                                       save_path,
                                       base_filename,
                                       bin_size):
    
    def add_zero_user(df):

        df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
        df['user_id_n'] = df['user_id_n'].astype(int)

        df.loc[-1] = ['u_1', 'i_1', ts, 0]
        df.loc[-4] = ['u_1', 'i_5', ts, 0]
        # df.loc[-2] = ['u_-1', 'i_2', ts, 0]
        # df.loc[-3] = ['u_-1', 'i_3', ts, 0]
        # df.loc[-5] = ['u_0', 'i_4', ts, 0]
        df.loc[-6] = ['u_1', 'drifted_i_1', ts, 0]
        df.loc[-7] = ['u_1', 'drifted_i_5', ts, 0]

        df.sort_values(by='user_id_n', inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.drop(columns=['user_id_n'], inplace=True)

        # print('added zero user\n', df.head())
        return df

    
    def create_folderpath(save_path, base_filename, specs_str):
        return save_path+base_filename+'_'+specs_str+'/'
    

    def save_items_frequencies(n_items_to_drift,
                               sudden_drift_start,
                               drift_items_freq_list,
                               non_drift_items_freq_list,
                               save_path, 
                               base_filename,
                               specs_str):
        
        folderpath = create_folderpath(save_path, base_filename, specs_str)
        validate_folderpath(folderpath)

        d = {'n_items_to_drift': n_items_to_drift,
             'sudden_drift_start': sudden_drift_start,
             'drift_items_freq_list': drift_items_freq_list,
             'non_drift_items_freq_list': non_drift_items_freq_list}

        save_picklefile(d, folderpath+'saved_dictionary.pkl')
       

    
    def save_dataset_atomic_file(df, user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())


    def add_user0_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)
            df_sampled = add_zero_user(df_sampled)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())




    def split_dataset_into_4_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

        
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def split_dataset_into_4_add_user0_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        add_user0_save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

                
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def calculate_sparsity(df):
        # df.item_id.groupby([df.user_id, df.item_id]).count().sum() == df.user_id.count()
        sparsity = 1 - df.user_id.count()/(df.user_id.nunique()*df.item_id.nunique())
        specs_str = str(df.user_id.nunique())+'x'+str(df.item_id.nunique())+'_'+str(round(sparsity, 2))
        print('specs_str', specs_str)
        return sparsity, specs_str
    

    def rename_item(row):
        if int(row['user_id'].split('_')[1]) > sudden_drift_start and row['item_id'] in renamed_items:
            return renamed_items[row['item_id']]
        return row['item_id']




    users_list = [f'u_{i+1}' for i in range(1, n_users)]
    items_list = [f'i_{j+1}' for j in range(n_items)]


    if all_items_seen:

        data = []
        for user in users_list:
            for item in items_list:
                data.append({'user_id': user, 'item_id': item, 'timestamp':ts})

        all_items_seen_df = pd.DataFrame(data)

        # Introduce sudden drift
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}       
        

        all_items_seen_df['item_id'] = all_items_seen_df.apply(rename_item, axis=1)
        # print(all_items_seen_df.item_id.groupby([all_items_seen_df.user_id, all_items_seen_df.item_id]).count().unstack().fillna(0).astype(int))

        
        sparsity , specs_str = calculate_sparsity(all_items_seen_df)
        print('sparsity: ',sparsity)

        sampled_df = add_zero_user(sampled_df)
        users_list.insert(0, 'u_0')
        

        # split_dataset_into_4_and_save_atomic_file(all_items_seen_df,users_list, bin_size, save_path, specs_str)
        # save_dataset_atomic_file(all_items_seen_df, save_path, specs_str)
        split_dataset_into_4_add_user0_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)
        

    else:
        
        if len(drift_items_freq_list) != n_items_to_drift:
            print('Not all items frequency was specified!')
            return None
        elif len(non_drift_items_freq_list) != len(items_list)-n_items_to_drift:
            print('Not all items frequency was specified!')
            return None


        def sample_with_repetition_of_pattern(users_list, items_list, items_freq_list):
            random.seed(random_seed)
            sampled_df = pd.DataFrame({})
            for i, freq in enumerate(items_freq_list):
                # print('k ',k)
                user_sample = random.sample(users_list[:sudden_drift_start], k=freq) +\
                                random.sample(users_list[sudden_drift_start:], k=freq)
                temp_df = pd.DataFrame({'user_id': user_sample, 
                                        'item_id': items_list[i]})
                # print(temp_df.item_id.groupby([temp_df.user_id, temp_df.item_id]).count().unstack().fillna(0).astype(int))
                sampled_df = pd.concat([sampled_df, temp_df])
            
            return sampled_df
        

        # Introduce sudden drift
        # No need to random sample, bc the list will have the frequencies for each item
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        # drift_items_list = [items_list[i] for i,x in enumerate(items_freq_list) if x == sudden_drift_start]
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}
        non_drift_items_list = list(set(items_list) - set(drift_items_list))
        
        print('drift_items_list', drift_items_list)
        print('renamed_items', renamed_items)
        print('non_drift_items_list', non_drift_items_list)

        
        random.seed(random_seed)  # For reproducibility
        sampled_df = sample_with_repetition_of_pattern(users_list,
                                                       non_drift_items_list,
                                                       non_drift_items_freq_list)
        
        sampled_df = pd.concat([sampled_df,
                                sample_with_repetition_of_pattern(users_list,
                                                                    drift_items_list,
                                                                    drift_items_freq_list)])

        if sampled_df.user_id.nunique() < n_users:
            # print(sampled_df.head())
            users_not_sampled = list(set(users_list) - set(sampled_df.user_id))
            print('users_not_sampled', len(users_not_sampled))
            # print('drift_items_list', drift_items_list)
            for user in users_not_sampled:
                for item in drift_items_list:
                    # print(sampled_df.loc[sampled_df['user_id']==user, 'item_id'].count())
                    sampled_df.loc[len(sampled_df)] = [user, item]


        sampled_df['item_id'] = sampled_df.apply(rename_item, axis=1)
        # print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))

        sampled_df['timestamp'] = ts



        sparsity, specs_str = calculate_sparsity(sampled_df)
        print('sparsity: ',sparsity)
        # print(specs_str)
        print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))
        # print(sampled_df.head())


        # when trainning on pt1, yield ValueError: Some users have interacted with all items, which we can not sample negative items for them. Please set `user_inter_num_interval` to filter those users.
        # sampled_df = add_zero_user(sampled_df) # to solve the error 
        users_list.insert(0, 'u_0')

        # save_dataset_atomic_file(sampled_df, save_path, specs_str)
        # split_dataset_into_4_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        split_dataset_into_4_add_user0_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)

        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)


        return sampled_df

# Parameters
n_users = 4000 # bc of random sample, some users dont have occurrencies
n_items = 5
sudden_drift_start = 2000  # Starting user index for drift (1-indexed)
bin_size = 1000

string = "24/12/2024 21:12:24"
ts = time.mktime(datetime.strptime(string, "%d/%m/%Y %H:%M:%S").timetuple())

all_items_seen = False

random_seed = 42

ratio_to_drift = 2 # Select 50% of items to rename
n_items_to_drift = n_items // ratio_to_drift

base_filename = 'sudden_drift_dataset_u0_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

drift_items_freq_list = [sudden_drift_start//2, sudden_drift_start//2]
non_drift_items_freq_list = [sudden_drift_start//4, sudden_drift_start//4, sudden_drift_start//4]

df = generate_artificial_random_dataset(n_users=n_users,
                                    n_items=n_items, 
                                    ts=ts,
                                    all_items_seen=all_items_seen,
                                    n_items_to_drift=n_items_to_drift,
                                    random_seed=random_seed,
                                    sudden_drift_start=sudden_drift_start,
                                    drift_items_freq_list=drift_items_freq_list,
                                    non_drift_items_freq_list=non_drift_items_freq_list,
                                    save_path=save_path,
                                    base_filename=base_filename,
                                    bin_size=bin_size)

## train test functions

In [1]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import BPR
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger


def train_test(model_name,
               dataset_name,
               parameter_dict):

    # configurations initialization
    config = Config(model=model_name, dataset=dataset_name, config_dict=parameter_dict)

    # init random seed
    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    model = BPR(config, train_data.dataset).to(config['device'])
    logger.info(model)

    # trainer loading and initialization
    trainer = Trainer(config, model)

    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    print('\n\nTraining best results')
    print('best_valid_score: ', best_valid_score)
    print('best_valid_result: ', best_valid_result)

    # model evaluation
    test_result = trainer.evaluate(test_data)
    
    print('\n\nTest results')
    print(test_result)

## train test

In [6]:
base_filename = 'sudden_drift_dataset_u0_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

# Current model~~~~
base_dataset_name = base_filename+'_3999x7_0.71'
data_path = 'processed_datasets/artificial_data/'

In [7]:
for part in ['_pt1', '_pt2', '_pt3', '']:
    print('\n\n'+part)
    dataset_name=base_dataset_name+part
    parameter_dict = {
        'dataset': dataset_name+'.inter',
        'data_path': data_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        'valid_metric':VALID_METRIC,
        'checkpoint_dir':data_path+dataset_name,
        'seed':SEED,
        'shuffle': SHUFFLE
    }


    train_test(MODEL, dataset_name, parameter_dict)

14 Jan 11:24    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_u0_i1i5_drift_all_parts_3999x7_0.71_pt1
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_u0_i1i5_drift_all_parts_3999x7_0.71_pt1
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', '



_pt1


 [0,inf)
alias_of_user_id = None
alias_of_item_id = None
alias_of_entity_id = None
alias_of_relation_id = None
preload_weight = None
normalize_field = None
normalize_all = None
ITEM_LIST_LENGTH_FIELD = item_length
LIST_SUFFIX = _list
MAX_ITEM_LIST_LENGTH = 50
POSITION_FIELD = position_id
HEAD_ENTITY_ID_FIELD = head_id
TAIL_ENTITY_ID_FIELD = tail_id
RELATION_ID_FIELD = relation_id
ENTITY_ID_FIELD = entity_id
benchmark_filename = None

Other Hyper Parameters: 
worker = 0
wandb_project = recbole
shuffle = False
require_pow = False
enable_amp = False
enable_scaler = False
transform = None
embedding_size = 64
numerical_features = []
discretization = None
kg_reverse_r = False
entity_kg_num_interval = [0,inf)
relation_kg_num_interval = [0,inf)
MODEL_TYPE = ModelType.GENERAL
MODEL_INPUT_TYPE = InputType.PAIRWISE
eval_type = EvaluatorType.RANKING
single_spec = True
local_rank = 0
device = cpu
valid_neg_sample_args = {'distribution': 'uniform', 'sample_num': 'none'}
test_neg_sample_args = {'dist

User ID Range: 1 to 1000
Embedding Weight Shape: torch.Size([1001, 64])


14 Jan 11:25    INFO  epoch 0 training [time: 0.13s, train loss: 0.6925]


...

User ID Range: 1 to 2030
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 2034 to 3999
Embedding Weight Shape: torch.Size([4001, 64])


14 Jan 11:25    INFO  epoch 11 evaluating [time: 0.14s, valid_score: 0.537200]
14 Jan 11:25    INFO  valid result: 
recall@3 : 0.5372    mrr@3 : 0.3185    ndcg@3 : 0.3742    hit@3 : 0.5372    precision@3 : 0.1791
14 Jan 11:25    INFO  Finished training, best eval result in epoch 0




Training best results
best_valid_score:  0.551
best_valid_result:  OrderedDict([('recall@3', 0.551), ('mrr@3', 0.3307), ('ndcg@3', 0.3869), ('hit@3', 0.551), ('precision@3', 0.1837)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load

User ID Range: 1 to 688
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 690 to 1425
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 1426 to 2188
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 2189 to 2886
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 2890 to 3579
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 3580 to 4000
Embedding Weight Shape: torch.Size([4001, 64])


Test results
OrderedDict([('recall@3', 0.5662), ('mrr@3', 0.3421), ('ndcg@3', 0.3992), ('hit@3', 0.5662), ('precision@3', 0.1887)])


## trigger error

In [8]:
def test_on_earlier_data(model_name,
                         earlier_datasets,
                         current_dataset_name,
                         parameter_dict,
                         checkpoint_file):


    current_config,\
        current_logger,\
                current_dataset,\
                    current_train_data, current_valid_data, current_test_data = setup_config_and_dataset(model_name,
                                                                                current_dataset_name,
                                                                                parameter_dict)

    # model loading and initialization
    current_model = BPR(current_config, current_train_data.dataset).to(current_config['device'])
    current_logger.info(current_model)


    # trainer loading and initialization
    trainer = get_trainer(current_config['MODEL_TYPE'], current_config['model'])(current_config, current_model)


    # results = []

    for earlier_dataset_name in earlier_datasets:
        print('\n\n'+earlier_dataset_name)
        earlier_config,\
        earlier_logger,\
                earlier_dataset,\
                    earlier_train_data, earlier_valid_data, earlier_test_data = setup_config_and_dataset(model_name,
                                                                                earlier_dataset_name,
                                                                                parameter_dict)


        # When calculate ItemCoverage metrics, we need to run this code for set item_nums in eval_collector.
        # trainer.eval_collector.data_collect(earlier_train_data)
        trainer.eval_collector.data_collect(current_train_data)

        # model evaluation
        test_result = trainer.evaluate(earlier_test_data, model_file=checkpoint_file)
        # results += [test_result]
    
        print(test_result)


def trigger_error(data_path, base_dataset_name, model_ver, checkpoint_ver, data_ver):
    current_ver = model_ver
    current_dataset_name = base_dataset_name+current_ver

    earlier_datasets = [base_dataset_name+data_ver]#,base_dataset_name+'_pt6', base_dataset_name+'_pt7', base_dataset_name+'_pt8'] # 1,5,6,7,8


    # Checkpoint - 
    # checkpoint_ver = 'BPR-Jan-01-2025_16-20-32'
    checkpoint_dir = data_path+base_dataset_name+current_ver
    checkpoint_file = checkpoint_dir+'/'+checkpoint_ver+'.pth'


    parameter_dict = {
        'dataset': current_dataset_name+'.inter',
        'data_path': data_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        'valid_metric':VALID_METRIC,
        # 'user_inter_num_interval':'[1,inf)',
        'checkpoint_dir':checkpoint_dir,
        'seed':SEED,
        'shuffle': SHUFFLE,
        'state':'ERROR',
        # 'show_progress': SHOW_PROGRESS,
        # https://recbole.io/docs/user_guide/config/evaluation_settings.html
        'eval_args': {'split': {'LS': 'test_only'}, # leave-one-out sample type
                    'group_by': 'user',
                    'order': 'RO', # random ordering
                    'mode': 'pop001'} #  for example pop100, means sample 100 negative items for each positive item in testing set based on item popularity (Counter(item) in .inter file), and evaluate the model on these positive items with their sampled negative items
    }

    test_on_earlier_data(MODEL,
                    earlier_datasets,
                    current_dataset_name,
                    parameter_dict, 
                    checkpoint_file)

### trigger_error(model_ver='_pt1', data_ver='_pt2') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-14-2025_11-24-59', data_ver='_pt2')

# re-generate data with added all users seeing item 1 at the beginning (trying to solve index out of range)

In [21]:
import os
import pandas as pd
import random
from datetime import datetime
import time
import pickle


def validate_folderpath(folderpath):
    # print('im validating')
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)
        print('Folder created: ', folderpath)


def save_picklefile(d, filepath):
    with open(filepath, 'wb') as f:
            pickle.dump(d, f)

    print('Saved file at '+filepath)

def load_picklefile(filepath):
    with open(filepath, 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

    


def generate_artificial_random_dataset(n_users,
                                       n_items, 
                                       ts,
                                       all_items_seen,
                                       random_seed,
                                       n_items_to_drift,
                                       sudden_drift_start,
                                       drift_items_freq_list,
                                       non_drift_items_freq_list,
                                       save_path,
                                       base_filename,
                                       bin_size):
    
    def add_zero_user(df):

        df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
        df['user_id_n'] = df['user_id_n'].astype(int)

        df.loc[-1] = ['u_0', 'i_1', ts, 0]
        df.loc[-4] = ['u_0', 'i_5', ts, 0]
        # df.loc[-2] = ['u_-1', 'i_2', ts, 0]
        # df.loc[-3] = ['u_-1', 'i_3', ts, 0]
        # df.loc[-5] = ['u_0', 'i_4', ts, 0]
        df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
        df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

        df.sort_values(by='user_id_n', inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.drop(columns=['user_id_n'], inplace=True)

        # print('added zero user\n', df.head())
        return df

    
    def create_folderpath(save_path, base_filename, specs_str):
        return save_path+base_filename+'_'+specs_str+'/'
    

    def save_items_frequencies(n_items_to_drift,
                               sudden_drift_start,
                               drift_items_freq_list,
                               non_drift_items_freq_list,
                               save_path, 
                               base_filename,
                               specs_str):
        
        folderpath = create_folderpath(save_path, base_filename, specs_str)
        validate_folderpath(folderpath)

        d = {'n_items_to_drift': n_items_to_drift,
             'sudden_drift_start': sudden_drift_start,
             'drift_items_freq_list': drift_items_freq_list,
             'non_drift_items_freq_list': non_drift_items_freq_list}

        save_picklefile(d, folderpath+'saved_dictionary.pkl')
       

    
    def save_dataset_atomic_file(df, user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())


    def add_user0_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)
            df_sampled = add_zero_user(df_sampled)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())

    

    # add all users to the beginning of the dataset
    def add_all_users_at_start(df, users_list):    
        df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
        df['user_id_n'] = df['user_id_n'].astype(int)

        # add all users, but user 0
        for i in range(1, n_users):
            df.loc[-i] = [users_list[i], 'i_1', ts, 0]
        df.reset_index(drop=True, inplace=True)

        # add user 0
        df.loc[-1] = ['u_0', 'i_1', ts, 0]
        df.loc[-4] = ['u_0', 'i_5', ts, 0]
        df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
        df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

        df.sort_values(by='user_id_n', inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.drop(columns=['user_id_n'], inplace=True)

        return df
    

    def add_all_users_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
        df = add_all_users_at_start(df, users_list)
        save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str)



    def split_dataset_into_4_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

        
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def split_dataset_into_4_add_user0_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        add_user0_save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

                
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')



    def split_dataset_into_4_add_users_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        add_all_users_save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
        add_all_users_save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        add_user0_save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

                
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')


    def calculate_sparsity(df):
        # df.item_id.groupby([df.user_id, df.item_id]).count().sum() == df.user_id.count()
        sparsity = 1 - df.user_id.count()/(df.user_id.nunique()*df.item_id.nunique())
        specs_str = str(df.user_id.nunique())+'x'+str(df.item_id.nunique())+'_'+str(round(sparsity, 2))
        print('specs_str', specs_str)
        return sparsity, specs_str
    

    def rename_item(row):
        if int(row['user_id'].split('_')[1]) > sudden_drift_start and row['item_id'] in renamed_items:
            return renamed_items[row['item_id']]
        return row['item_id']




    # users_list = [f'u_{i+1}' for i in range(1, n_users)]
    users_list = [f'u_{i+1}' for i in range(n_users)]
    items_list = [f'i_{j+1}' for j in range(n_items)]


    if all_items_seen:

        data = []
        for user in users_list:
            for item in items_list:
                data.append({'user_id': user, 'item_id': item, 'timestamp':ts})

        all_items_seen_df = pd.DataFrame(data)

        # Introduce sudden drift
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}       
        

        all_items_seen_df['item_id'] = all_items_seen_df.apply(rename_item, axis=1)
        # print(all_items_seen_df.item_id.groupby([all_items_seen_df.user_id, all_items_seen_df.item_id]).count().unstack().fillna(0).astype(int))

        
        sparsity , specs_str = calculate_sparsity(all_items_seen_df)
        print('sparsity: ',sparsity)

        sampled_df = add_zero_user(sampled_df)
        users_list.insert(0, 'u_0')
        

        # split_dataset_into_4_and_save_atomic_file(all_items_seen_df,users_list, bin_size, save_path, specs_str)
        # save_dataset_atomic_file(all_items_seen_df, save_path, specs_str)
        split_dataset_into_4_add_users_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)
        

    else:
        
        if len(drift_items_freq_list) != n_items_to_drift:
            print('Not all items frequency was specified!')
            return None
        elif len(non_drift_items_freq_list) != len(items_list)-n_items_to_drift:
            print('Not all items frequency was specified!')
            return None


        def sample_with_repetition_of_pattern(users_list, items_list, items_freq_list):
            random.seed(random_seed)
            sampled_df = pd.DataFrame({})
            for i, freq in enumerate(items_freq_list):
                # print('k ',k)
                user_sample = random.sample(users_list[:sudden_drift_start], k=freq) +\
                                random.sample(users_list[sudden_drift_start:], k=freq)
                temp_df = pd.DataFrame({'user_id': user_sample, 
                                        'item_id': items_list[i]})
                # print(temp_df.item_id.groupby([temp_df.user_id, temp_df.item_id]).count().unstack().fillna(0).astype(int))
                sampled_df = pd.concat([sampled_df, temp_df])
            
            return sampled_df
        

        # Introduce sudden drift
        # No need to random sample, bc the list will have the frequencies for each item
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        # drift_items_list = [items_list[i] for i,x in enumerate(items_freq_list) if x == sudden_drift_start]
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}
        non_drift_items_list = list(set(items_list) - set(drift_items_list))
        
        print('drift_items_list', drift_items_list)
        print('renamed_items', renamed_items)
        print('non_drift_items_list', non_drift_items_list)

        
        random.seed(random_seed)  # For reproducibility
        sampled_df = sample_with_repetition_of_pattern(users_list,
                                                       non_drift_items_list,
                                                       non_drift_items_freq_list)
        
        sampled_df = pd.concat([sampled_df,
                                sample_with_repetition_of_pattern(users_list,
                                                                    drift_items_list,
                                                                    drift_items_freq_list)])

        if sampled_df.user_id.nunique() < n_users:
            # print(sampled_df.head())
            users_not_sampled = list(set(users_list) - set(sampled_df.user_id))
            print('users_not_sampled', len(users_not_sampled))
            # print('drift_items_list', drift_items_list)
            for user in users_not_sampled:
                for item in drift_items_list:
                    # print(sampled_df.loc[sampled_df['user_id']==user, 'item_id'].count())
                    sampled_df.loc[len(sampled_df)] = [user, item]


        sampled_df['item_id'] = sampled_df.apply(rename_item, axis=1)
        # print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))

        sampled_df['timestamp'] = ts


        sparsity, specs_str = calculate_sparsity(sampled_df)
        print('sparsity: ',sparsity)
        # print(specs_str)
        print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))
        # print(sampled_df.head())


        # when trainning on pt1, yield ValueError: Some users have interacted with all items, which we can not sample negative items for them. Please set `user_inter_num_interval` to filter those users.
        # sampled_df = add_zero_user(sampled_df) # to solve the error 
        users_list.insert(0, 'u_0')

        # save_dataset_atomic_file(sampled_df, save_path, specs_str)
        # split_dataset_into_4_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        split_dataset_into_4_add_users_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)

        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)


        return sampled_df

# Parameters
n_users = 4000 # bc of random sample, some users dont have occurrencies
n_items = 5
sudden_drift_start = 2000  # Starting user index for drift (1-indexed)
bin_size = 1000

string = "24/12/2024 21:12:24"
ts = time.mktime(datetime.strptime(string, "%d/%m/%Y %H:%M:%S").timetuple())

all_items_seen = False

random_seed = 42

ratio_to_drift = 2 # Select 50% of items to rename
n_items_to_drift = n_items // ratio_to_drift

base_filename = 'sudden_drift_dataset_all_users_start_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

drift_items_freq_list = [sudden_drift_start//2, sudden_drift_start//2]
non_drift_items_freq_list = [sudden_drift_start//4, sudden_drift_start//4, sudden_drift_start//4]

df = generate_artificial_random_dataset(n_users=n_users,
                                    n_items=n_items, 
                                    ts=ts,
                                    all_items_seen=all_items_seen,
                                    n_items_to_drift=n_items_to_drift,
                                    random_seed=random_seed,
                                    sudden_drift_start=sudden_drift_start,
                                    drift_items_freq_list=drift_items_freq_list,
                                    non_drift_items_freq_list=non_drift_items_freq_list,
                                    save_path=save_path,
                                    base_filename=base_filename,
                                    bin_size=bin_size)

drift_items_list ['i_1', 'i_5']


renamed_items {'i_1': 'drifted_i_1', 'i_5': 'drifted_i_5'}
non_drift_items_list ['i_3', 'i_2', 'i_4']
users_not_sampled 571
specs_str 4000x7_0.71
sparsity:  0.7092142857142858
item_id  drifted_i_1  drifted_i_5  i_1  i_2  i_3  i_4  i_5
user_id                                                   
u_1                0            0    1    0    0    0    0
u_10               0            0    1    0    0    0    1
u_100              0            0    0    0    0    0    1
u_1000             0            0    1    1    0    0    0
u_1001             0            0    1    1    1    0    1
...              ...          ...  ...  ...  ...  ...  ...
u_995              0            0    1    0    1    0    0
u_996              0            0    1    0    1    1    0
u_997              0            0    1    0    1    0    0
u_998              0            0    1    0    0    0    1
u_999              0            0    1    1    0    0    0

[4000 rows x 7 columns]
  user_id item_id     timestamp

## train test functions

In [44]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import BPR
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger


def train_test(model_name,
               dataset_name,
               parameter_dict):

    # configurations initialization
    config = Config(model=model_name, dataset=dataset_name, config_dict=parameter_dict)

    # init random seed
    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    model = BPR(config, train_data.dataset).to(config['device'])
    logger.info(model)

    # trainer loading and initialization
    trainer = Trainer(config, model)

    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    print('\n\nTraining best results')
    print('best_valid_score: ', best_valid_score)
    print('best_valid_result: ', best_valid_result)

    # model evaluation
    test_result = trainer.evaluate(test_data)
    
    print('\n\nTest results')
    print(test_result)

## train test Updated all users at start splittage version

In [45]:
base_filename = 'sudden_drift_dataset_all_users_start_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

# Current model
base_dataset_name = base_filename+'_4000x7_0.71'
data_path = 'processed_datasets/artificial_data/'

In [46]:
for part in ['_pt1', '_pt2', '_pt3', '']:
    print('\n\n'+part)
    dataset_name=base_dataset_name+part
    parameter_dict = {
        'dataset': dataset_name+'.inter',
        'data_path': data_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        'valid_metric':VALID_METRIC,
        'checkpoint_dir':data_path+dataset_name,
        'seed':SEED,
        'shuffle': SHUFFLE
    }


    train_test(MODEL, dataset_name, parameter_dict)



_pt1


14 Jan 17:34    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = INFO
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_all_users_start_i1i5_drift_all_parts_4000x7_0.71_pt1
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_all_users_start_i1i5_drift_all_parts_4000x7_0.71_pt1
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = Fa

User ID Range: 1 to 3999
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 2 to 4000
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 9 to 3770
Embedding Weight Shape: torch.Size([4001, 64])


14 Jan 17:34    INFO  epoch 0 training [time: 0.42s, train loss: 2.0786]


User ID Range: 996 to 3720
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 3721 to 4000
Embedding Weight Shape: torch.Size([4001, 64])


...

User ID Range: 996 to 3720
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 3721 to 4000
Embedding Weight Shape: torch.Size([4001, 64])


14 Jan 17:36    INFO  epoch 19 evaluating [time: 0.42s, valid_score: 0.615100]
14 Jan 17:36    INFO  valid result: 
recall@3 : 0.6151    mrr@3 : 0.4898    ndcg@3 : 0.5217    hit@3 : 0.6151    precision@3 : 0.205
14 Jan 17:36    INFO  Finished training, best eval result in epoch 8




Training best results
best_valid_score:  0.6806
best_valid_result:  OrderedDict([('recall@3', 0.6806), ('mrr@3', 0.5074), ('ndcg@3', 0.5516), ('hit@3', 0.6806), ('precision@3', 0.2269)])


c:\Users\mjlav\anaconda3\envs\algorithms_transparency\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load

User ID Range: 1 to 3512
Embedding Weight Shape: torch.Size([4001, 64])
User ID Range: 3513 to 4000
Embedding Weight Shape: torch.Size([4001, 64])


Test results
OrderedDict([('recall@3', 0.687), ('mrr@3', 0.516), ('ndcg@3', 0.5597), ('hit@3', 0.687), ('precision@3', 0.229)])


## trigger error

In [47]:
def test_on_earlier_data(model_name,
                         earlier_datasets,
                         current_dataset_name,
                         parameter_dict,
                         checkpoint_file):


    current_config,\
        current_logger,\
                current_dataset,\
                    current_train_data, current_valid_data, current_test_data = setup_config_and_dataset(model_name,
                                                                                current_dataset_name,
                                                                                parameter_dict)

    # model loading and initialization
    current_model = BPR(current_config, current_train_data.dataset).to(current_config['device'])
    current_logger.info(current_model)


    # trainer loading and initialization
    trainer = get_trainer(current_config['MODEL_TYPE'], current_config['model'])(current_config, current_model)


    # results = []

    for earlier_dataset_name in earlier_datasets:
        print('\n\n'+earlier_dataset_name)
        earlier_config,\
        earlier_logger,\
                earlier_dataset,\
                    earlier_train_data, earlier_valid_data, earlier_test_data = setup_config_and_dataset(model_name,
                                                                                earlier_dataset_name,
                                                                                parameter_dict)


        # When calculate ItemCoverage metrics, we need to run this code for set item_nums in eval_collector.
        # trainer.eval_collector.data_collect(earlier_train_data)
        trainer.eval_collector.data_collect(current_train_data)

        # model evaluation
        test_result = trainer.evaluate(earlier_test_data, model_file=checkpoint_file)
        # results += [test_result]
    
        print(test_result)


def trigger_error(data_path, base_dataset_name, model_ver, checkpoint_ver, data_ver):
    current_ver = model_ver
    current_dataset_name = base_dataset_name+current_ver

    earlier_datasets = [base_dataset_name+data_ver]#,base_dataset_name+'_pt6', base_dataset_name+'_pt7', base_dataset_name+'_pt8'] # 1,5,6,7,8


    # Checkpoint - 
    # checkpoint_ver = 'BPR-Jan-01-2025_16-20-32'
    checkpoint_dir = data_path+base_dataset_name+current_ver
    checkpoint_file = checkpoint_dir+'/'+checkpoint_ver+'.pth'


    parameter_dict = {
        'dataset': current_dataset_name+'.inter',
        'data_path': data_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        'valid_metric':VALID_METRIC,
        # 'user_inter_num_interval':'[1,inf)',
        'checkpoint_dir':checkpoint_dir,
        'seed':SEED,
        'shuffle': SHUFFLE,
        'state':'ERROR',
        # 'show_progress': SHOW_PROGRESS,
        # https://recbole.io/docs/user_guide/config/evaluation_settings.html
        'eval_args': {'split': {'LS': 'test_only'}, # leave-one-out sample type
                    'group_by': 'user',
                    'order': 'RO', # random ordering
                    'mode': 'pop001'} #  for example pop100, means sample 100 negative items for each positive item in testing set based on item popularity (Counter(item) in .inter file), and evaluate the model on these positive items with their sampled negative items
    }

    test_on_earlier_data(MODEL,
                    earlier_datasets,
                    current_dataset_name,
                    parameter_dict, 
                    checkpoint_file)

### trigger_error(model_ver='_pt1', data_ver='_pt2') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-13-2025_17-12-22', data_ver='_pt2')

## checking whether the data was properly constructed

In [29]:
base_filename = 'sudden_drift_dataset_all_users_start_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

# Current model
base_dataset_name = base_filename+'_4000x7_0.71'
data_path = 'processed_datasets/artificial_data/'

In [30]:
df_pt1 = pd.read_csv(get_filepath(save_path, base_dataset_name, '_pt1')+'.csv')
df_pt1

,user_id,item_id,timestamp
0,u_0,drifted_i_5,1.735093e+09
1,u_999,i_1,1.735093e+09
2,u_341,i_1,1.735093e+09
3,u_340,i_1,1.735093e+09
4,u_339,i_1,1.735093e+09
...,...,...,...
4062,u_998,i_5,1.735093e+09
4063,u_998,i_1,1.735093e+09
4064,u_999,i_2,1.735093e+09
4065,u_999,i_1,1.735093e+09


In [32]:
df_pt1.user_id.nunique()

1000

In [34]:
df_full = pd.read_csv(get_filepath(save_path, base_dataset_name, '')+'.csv')
df_full.user_id.nunique()

4001

clearly it did not add all users to 1st part

In [42]:
import os
import pandas as pd
import random
from datetime import datetime
import time
import pickle


# Parameters
n_users = 4000 # bc of random sample, some users dont have occurrencies
n_items = 5
sudden_drift_start = 2000  # Starting user index for drift (1-indexed)
bin_size = 1000

string = "24/12/2024 21:12:24"
ts = time.mktime(datetime.strptime(string, "%d/%m/%Y %H:%M:%S").timetuple())

all_items_seen = False

random_seed = 42

ratio_to_drift = 2 # Select 50% of items to rename
n_items_to_drift = n_items // ratio_to_drift

base_filename = 'sudden_drift_dataset_all_users_start_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

drift_items_freq_list = [sudden_drift_start//2, sudden_drift_start//2]
non_drift_items_freq_list = [sudden_drift_start//4, sudden_drift_start//4, sudden_drift_start//4]

def add_zero_user(df):

    df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
    df['user_id_n'] = df['user_id_n'].astype(int)

    df.loc[-1] = ['u_0', 'i_1', ts, 0]
    df.loc[-4] = ['u_0', 'i_5', ts, 0]
    # df.loc[-2] = ['u_-1', 'i_2', ts, 0]
    # df.loc[-3] = ['u_-1', 'i_3', ts, 0]
    # df.loc[-5] = ['u_0', 'i_4', ts, 0]
    df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
    df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

    df.sort_values(by='user_id_n', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.drop(columns=['user_id_n'], inplace=True)

    # print('added zero user\n', df.head())
    return df


def save_items_frequencies(n_items_to_drift,
                            sudden_drift_start,
                            drift_items_freq_list,
                            non_drift_items_freq_list,
                            save_path, 
                            base_filename,
                            specs_str):
    
    folderpath = create_folderpath(save_path, base_filename, specs_str)
    validate_folderpath(folderpath)

    d = {'n_items_to_drift': n_items_to_drift,
            'sudden_drift_start': sudden_drift_start,
            'drift_items_freq_list': drift_items_freq_list,
            'non_drift_items_freq_list': non_drift_items_freq_list}

    save_picklefile(d, folderpath+'saved_dictionary.pkl')
    


def save_dataset_atomic_file(df, user_sample, save_path, base_filename, specs_str):
    if save_path:
        folderpath = create_folderpath(save_path, base_filename, specs_str)
        validate_folderpath(folderpath)
        # Output the dataset
        filepath = folderpath+base_filename+'_'+specs_str

        df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)

        df_sampled.to_csv(filepath+'.csv', index=False)
        df_sampled.to_csv(filepath+'.inter',
                            header=['user_id:token','item_id:token','timestamp:float'], 
                            sep='\t', 
                            index=False)
        print("Dataset with sudden drift created and saved at "+filepath+".")
        print(df_sampled.item_id.value_counts())


def add_user0_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
    if save_path:
        folderpath = create_folderpath(save_path, base_filename, specs_str)
        validate_folderpath(folderpath)
        # Output the dataset
        filepath = folderpath+base_filename+'_'+specs_str

        df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)
        df_sampled = add_zero_user(df_sampled)

        df_sampled.to_csv(filepath+'.csv', index=False)
        df_sampled.to_csv(filepath+'.inter',
                            header=['user_id:token','item_id:token','timestamp:float'], 
                            sep='\t', 
                            index=False)
        print("Dataset with sudden drift created and saved at "+filepath+".")
        print(df_sampled.item_id.value_counts())



def calculate_sparsity(df):
    # df.item_id.groupby([df.user_id, df.item_id]).count().sum() == df.user_id.count()
    sparsity = 1 - df.user_id.count()/(df.user_id.nunique()*df.item_id.nunique())
    specs_str = str(df.user_id.nunique())+'x'+str(df.item_id.nunique())+'_'+str(round(sparsity, 2))
    print('specs_str', specs_str)
    return sparsity, specs_str


def rename_item(row):
    if int(row['user_id'].split('_')[1]) > sudden_drift_start and row['item_id'] in renamed_items:
        return renamed_items[row['item_id']]
    return row['item_id']

def create_folderpath(save_path, base_filename, specs_str):
    return save_path+base_filename+'_'+specs_str+'/'


def sample_with_repetition_of_pattern(users_list, items_list, items_freq_list):
    random.seed(random_seed)
    sampled_df = pd.DataFrame({})
    for i, freq in enumerate(items_freq_list):
        # print('k ',k)
        user_sample = random.sample(users_list[:sudden_drift_start], k=freq) +\
                        random.sample(users_list[sudden_drift_start:], k=freq)
        temp_df = pd.DataFrame({'user_id': user_sample, 
                                'item_id': items_list[i]})
        # print(temp_df.item_id.groupby([temp_df.user_id, temp_df.item_id]).count().unstack().fillna(0).astype(int))
        sampled_df = pd.concat([sampled_df, temp_df])
    
    return sampled_df




# users_list = [f'u_{i+1}' for i in range(1, n_users)]
users_list = [f'u_{i+1}' for i in range(n_users)]
items_list = [f'i_{j+1}' for j in range(n_items)]



# Introduce sudden drift
# No need to random sample, bc the list will have the frequencies for each item
random.seed(random_seed)  # For reproducibility
drift_items_list = random.sample(items_list, k=n_items_to_drift)  
# drift_items_list = [items_list[i] for i,x in enumerate(items_freq_list) if x == sudden_drift_start]
renamed_items = {item: f'drifted_{item}' for item in drift_items_list}
non_drift_items_list = list(set(items_list) - set(drift_items_list))

print('drift_items_list', drift_items_list)
print('renamed_items', renamed_items)
print('non_drift_items_list', non_drift_items_list)


random.seed(random_seed)  # For reproducibility
sampled_df = sample_with_repetition_of_pattern(users_list,
                                                non_drift_items_list,
                                                non_drift_items_freq_list)

sampled_df = pd.concat([sampled_df,
                        sample_with_repetition_of_pattern(users_list,
                                                            drift_items_list,
                                                            drift_items_freq_list)])

if sampled_df.user_id.nunique() < n_users:
    # print(sampled_df.head())
    users_not_sampled = list(set(users_list) - set(sampled_df.user_id))
    print('users_not_sampled', len(users_not_sampled))
    # print('drift_items_list', drift_items_list)
    for user in users_not_sampled:
        for item in drift_items_list:
            # print(sampled_df.loc[sampled_df['user_id']==user, 'item_id'].count())
            sampled_df.loc[len(sampled_df)] = [user, item]


sampled_df['item_id'] = sampled_df.apply(rename_item, axis=1)
# print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))

sampled_df['timestamp'] = ts


sparsity, specs_str = calculate_sparsity(sampled_df)
print('sparsity: ',sparsity)
# print(specs_str)
print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))
# print(sampled_df.head())


# when trainning on pt1, yield ValueError: Some users have interacted with all items, which we can not sample negative items for them. Please set `user_inter_num_interval` to filter those users.
# sampled_df = add_zero_user(sampled_df) # to solve the error 
users_list.insert(0, 'u_0')


# split_dataset_into_4_add_users_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)

## add_all_users_save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)

######################################################################################################################

    # def add_zero_user(df):

    #     df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
    #     df['user_id_n'] = df['user_id_n'].astype(int)

    #     df.loc[-1] = ['u_0', 'i_1', ts, 0]
    #     df.loc[-4] = ['u_0', 'i_5', ts, 0]
    #     # df.loc[-2] = ['u_-1', 'i_2', ts, 0]
    #     # df.loc[-3] = ['u_-1', 'i_3', ts, 0]
    #     # df.loc[-5] = ['u_0', 'i_4', ts, 0]
    #     df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
    #     df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

    #     df.sort_values(by='user_id_n', inplace=True)
    #     df.reset_index(drop=True, inplace=True)
    #     df.drop(columns=['user_id_n'], inplace=True)

    #     # print('added zero user\n', df.head())
    #     return df

    
    
    # def save_dataset_atomic_file(df, user_sample, save_path, base_filename, specs_str):
    #     if save_path:
    #         folderpath = create_folderpath(save_path, base_filename, specs_str)
    #         validate_folderpath(folderpath)
    #         # Output the dataset
    #         filepath = folderpath+base_filename+'_'+specs_str

    #         df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)

    #         df_sampled.to_csv(filepath+'.csv', index=False)
    #         df_sampled.to_csv(filepath+'.inter',
    #                             header=['user_id:token','item_id:token','timestamp:float'], 
    #                             sep='\t', 
    #                             index=False)
    #         print("Dataset with sudden drift created and saved at "+filepath+".")
    #         print(df_sampled.item_id.value_counts())


    # def add_user0_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
    #     if save_path:
    #         folderpath = create_folderpath(save_path, base_filename, specs_str)
    #         validate_folderpath(folderpath)
    #         # Output the dataset
    #         filepath = folderpath+base_filename+'_'+specs_str

    #         df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)
    #         df_sampled = add_zero_user(df_sampled)

    #         df_sampled.to_csv(filepath+'.csv', index=False)
    #         df_sampled.to_csv(filepath+'.inter',
    #                             header=['user_id:token','item_id:token','timestamp:float'], 
    #                             sep='\t', 
    #                             index=False)
    #         print("Dataset with sudden drift created and saved at "+filepath+".")
    #         print(df_sampled.item_id.value_counts())

    

    # def add_all_users_at_start(df, users_list):    
    #     df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
    #     df['user_id_n'] = df['user_id_n'].astype(int)

    #     # add all users, but user 0
    #     for i in range(1, n_users):
    #         df.loc[-i] = [users_list[i], 'i_1', ts, 0]
    #     df.reset_index(drop=True, inplace=True)

    #     # add user 0
    #     df.loc[-1] = ['u_0', 'i_1', ts, 0]
    #     df.loc[-4] = ['u_0', 'i_5', ts, 0]
    #     df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
    #     df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

    #     df.sort_values(by='user_id_n', inplace=True)
    #     df.reset_index(drop=True, inplace=True)
    #     df.drop(columns=['user_id_n'], inplace=True)

    #     return df
    

    # def add_all_users_save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str):
    #     df = add_all_users_at_start(df, users_list)
    #     save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str)



    # def split_dataset_into_4_add_users_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
    #     print(df.head())
    #     add_all_users_save_dataset_atomic_file(df, users_list, save_path, base_filename, specs_str)
    #     add_all_users_save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
    #     add_user0_save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
    #     add_user0_save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

                
    #     add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
    #     add_user0_save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
    #     add_user0_save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
    #     add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')





## add_all_users_save_dataset_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
print('User list range: '+str(min(users_list))+'-'+str(max(users_list))+'; len: '+str(len(users_list)))

user_sample = users_list[:bin_size]
df = sampled_df[sampled_df.user_id.isin(user_sample)].reset_index(drop=True)
print('Number of user ids in the dataset TO BE part_1: ', df.user_id.nunique())

df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
df['user_id_n'] = df['user_id_n'].astype(int)

# add all users, but user 0
for i in range(1, n_users):
    df.loc[-i] = [users_list[i], 'i_1', ts, 0]
df.reset_index(drop=True, inplace=True)

# add user 0
df.loc[-1] = ['u_0', 'i_1', ts, 0]
df.loc[-4] = ['u_0', 'i_5', ts, 0]
df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

df.sort_values(by='user_id_n', inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(columns=['user_id_n'], inplace=True)

print(df.head())
print('Number of user ids in the dataset part_1 (all users alredy added): ', df.user_id.nunique())
print('User list: ',df.user_id.unique())

### save_dataset_atomic_file(df,user_sample, save_path, base_filename, specs_str)
# folderpath = create_folderpath(save_path, base_filename, specs_str)
# validate_folderpath(folderpath)
# Output the dataset
# filepath = folderpath+base_filename+'_'+specs_str

# HERE IS THE PROBLEM! -> the df needs to be sampled 1st and then added all users to beginning
# df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)



# df_sampled.to_csv(filepath+'.csv', index=False)
# df_sampled.to_csv(filepath+'.inter',
#                     header=['user_id:token','item_id:token','timestamp:float'], 
#                     sep='\t', 
#                     index=False)
# print("Dataset with sudden drift created and saved at "+filepath+".")
# print(df_sampled.item_id.value_counts())


######################################################################################################################


## add_user0_save_dataset_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
## add_user0_save_dataset_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')


## add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
## add_user0_save_dataset_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
## add_user0_save_dataset_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
## add_user0_save_dataset_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')







# save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)



drift_items_list ['i_1', 'i_5']


renamed_items {'i_1': 'drifted_i_1', 'i_5': 'drifted_i_5'}
non_drift_items_list ['i_2', 'i_4', 'i_3']
users_not_sampled 571
specs_str 4000x7_0.71
sparsity:  0.7092142857142858
item_id  drifted_i_1  drifted_i_5  i_1  i_2  i_3  i_4  i_5
user_id                                                   
u_1                0            0    1    0    0    0    0
u_10               0            0    1    0    0    0    1
u_100              0            0    0    0    0    0    1
u_1000             0            0    1    0    0    1    0
u_1001             0            0    1    1    0    1    1
...              ...          ...  ...  ...  ...  ...  ...
u_995              0            0    1    1    0    0    0
u_996              0            0    1    1    1    0    0
u_997              0            0    1    1    0    0    0
u_998              0            0    1    0    0    0    1
u_999              0            0    1    0    0    1    0

[4000 rows x 7 columns]
User list range: u_0-u_999; len

### re-doing generating data functions

In [43]:
import os
import pandas as pd
import random
from datetime import datetime
import time
import pickle


def validate_folderpath(folderpath):
    # print('im validating')
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)
        print('Folder created: ', folderpath)


def save_picklefile(d, filepath):
    with open(filepath, 'wb') as f:
            pickle.dump(d, f)

    print('Saved file at '+filepath)

def load_picklefile(filepath):
    with open(filepath, 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

    


def generate_artificial_random_dataset(n_users,
                                       n_items, 
                                       ts,
                                       all_items_seen,
                                       random_seed,
                                       n_items_to_drift,
                                       sudden_drift_start,
                                       drift_items_freq_list,
                                       non_drift_items_freq_list,
                                       save_path,
                                       base_filename,
                                       bin_size):
    

    def add_zero_user(df):

        df['user_id_n'] = df['user_id'].apply(lambda x: x[2:])
        df['user_id_n'] = df['user_id_n'].astype(int)

        df.loc[-1] = ['u_0', 'i_1', ts, 0]
        df.loc[-4] = ['u_0', 'i_5', ts, 0]
        df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
        df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

        df.sort_values(by='user_id_n', inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.drop(columns=['user_id_n'], inplace=True)
        return df

    
    def create_folderpath(save_path, base_filename, specs_str):
        return save_path+base_filename+'_'+specs_str+'/'
    

    def save_items_frequencies(n_items_to_drift,
                               sudden_drift_start,
                               drift_items_freq_list,
                               non_drift_items_freq_list,
                               save_path, 
                               base_filename,
                               specs_str):
        
        folderpath = create_folderpath(save_path, base_filename, specs_str)
        validate_folderpath(folderpath)

        d = {'n_items_to_drift': n_items_to_drift,
             'sudden_drift_start': sudden_drift_start,
             'drift_items_freq_list': drift_items_freq_list,
             'non_drift_items_freq_list': non_drift_items_freq_list}

        save_picklefile(d, folderpath+'saved_dictionary.pkl')
       

    def save_complete_dataset_atomic_file(df, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df.to_csv(filepath+'.csv', index=False)
            df.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df.item_id.value_counts())



    def add_user0_save_dataset_sample_atomic_file(df,user_sample, save_path, base_filename, specs_str):
        if save_path:
            folderpath = create_folderpath(save_path, base_filename, specs_str)
            validate_folderpath(folderpath)
            # Output the dataset
            filepath = folderpath+base_filename+'_'+specs_str

            df_sampled = df[df.user_id.isin(user_sample)].reset_index(drop=True)
            df_sampled = add_zero_user(df_sampled)

            df_sampled.to_csv(filepath+'.csv', index=False)
            df_sampled.to_csv(filepath+'.inter',
                                header=['user_id:token','item_id:token','timestamp:float'], 
                                sep='\t', 
                                index=False)
            print("Dataset with sudden drift created and saved at "+filepath+".")
            print(df_sampled.item_id.value_counts())

    
    

    def add_all_users_save_dataset_sample_atomic_file(df,user_sample, save_path, base_filename, specs_str):
        user_sample = users_list[:bin_size]
        sampled_df = df[df.user_id.isin(user_sample)].reset_index(drop=True)
        # print('Number of user ids in the dataset TO BE part_1: ', sampled_df.user_id.nunique())

        sampled_df['user_id_n'] = sampled_df['user_id'].apply(lambda x: x[2:])
        sampled_df['user_id_n'] = sampled_df['user_id_n'].astype(int)

        # add all users, but user 0
        for i in range(1, n_users):
            sampled_df.loc[-i] = [users_list[i], 'i_1', ts, 0]
        sampled_df.reset_index(drop=True, inplace=True)

        # add user 0
        sampled_df.loc[-1] = ['u_0', 'i_1', ts, 0]
        sampled_df.loc[-4] = ['u_0', 'i_5', ts, 0]
        sampled_df.loc[-6] = ['u_0', 'drifted_i_1', ts, 0]
        sampled_df.loc[-7] = ['u_0', 'drifted_i_5', ts, 0]

        sampled_df.sort_values(by='user_id_n', inplace=True)
        sampled_df.reset_index(drop=True, inplace=True)
        sampled_df.drop(columns=['user_id_n'], inplace=True)

        print(sampled_df.head())
        print('Number of user ids in the dataset (all users alredy added): ', df.user_id.nunique())

        save_complete_dataset_atomic_file(sampled_df, save_path, base_filename, specs_str)


    def split_dataset_into_4_add_users_and_save_atomic_file(df, users_list, bin_size, save_path, base_filename, specs_str):
        print(df.head())
        add_all_users_save_dataset_sample_atomic_file(df, users_list, save_path, base_filename, specs_str)
        add_all_users_save_dataset_sample_atomic_file(df, users_list[:bin_size], save_path, base_filename, specs_str+'_pt1')
        add_user0_save_dataset_sample_atomic_file(df, users_list[:bin_size*2], save_path, base_filename, specs_str+'_pt2')
        add_user0_save_dataset_sample_atomic_file(df, users_list[:bin_size*3], save_path, base_filename, specs_str+'_pt3')

                
        add_user0_save_dataset_sample_atomic_file(df, users_list[bin_size:bin_size*2], save_path, base_filename, specs_str+'_pt5')
        add_user0_save_dataset_sample_atomic_file(df, users_list[bin_size*2:bin_size*3], save_path, base_filename, specs_str+'_pt6')
        add_user0_save_dataset_sample_atomic_file(df, users_list[bin_size*3:], save_path, base_filename, specs_str+'_pt7')
        add_user0_save_dataset_sample_atomic_file(df, users_list[bin_size:bin_size*3], save_path, base_filename, specs_str+'_pt8')


    def calculate_sparsity(df):
        # df.item_id.groupby([df.user_id, df.item_id]).count().sum() == df.user_id.count()
        sparsity = 1 - df.user_id.count()/(df.user_id.nunique()*df.item_id.nunique())
        specs_str = str(df.user_id.nunique())+'x'+str(df.item_id.nunique())+'_'+str(round(sparsity, 2))
        print('specs_str', specs_str)
        return sparsity, specs_str
    

    def rename_item(row):
        if int(row['user_id'].split('_')[1]) > sudden_drift_start and row['item_id'] in renamed_items:
            return renamed_items[row['item_id']]
        return row['item_id']




    # users_list = [f'u_{i+1}' for i in range(1, n_users)]
    users_list = [f'u_{i+1}' for i in range(n_users)]
    items_list = [f'i_{j+1}' for j in range(n_items)]


    if all_items_seen:

        data = []
        for user in users_list:
            for item in items_list:
                data.append({'user_id': user, 'item_id': item, 'timestamp':ts})

        all_items_seen_df = pd.DataFrame(data)

        # Introduce sudden drift
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}       
        

        all_items_seen_df['item_id'] = all_items_seen_df.apply(rename_item, axis=1)
        # print(all_items_seen_df.item_id.groupby([all_items_seen_df.user_id, all_items_seen_df.item_id]).count().unstack().fillna(0).astype(int))

        
        sparsity , specs_str = calculate_sparsity(all_items_seen_df)
        print('sparsity: ',sparsity)

        sampled_df = add_zero_user(sampled_df)
        users_list.insert(0, 'u_0')
        
        split_dataset_into_4_add_users_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)
        

    else:
        
        if len(drift_items_freq_list) != n_items_to_drift:
            print('Not all items frequency was specified!')
            return None
        elif len(non_drift_items_freq_list) != len(items_list)-n_items_to_drift:
            print('Not all items frequency was specified!')
            return None


        def sample_with_repetition_of_pattern(users_list, items_list, items_freq_list):
            random.seed(random_seed)
            sampled_df = pd.DataFrame({})
            for i, freq in enumerate(items_freq_list):
                # print('k ',k)
                user_sample = random.sample(users_list[:sudden_drift_start], k=freq) +\
                                random.sample(users_list[sudden_drift_start:], k=freq)
                temp_df = pd.DataFrame({'user_id': user_sample, 
                                        'item_id': items_list[i]})
                # print(temp_df.item_id.groupby([temp_df.user_id, temp_df.item_id]).count().unstack().fillna(0).astype(int))
                sampled_df = pd.concat([sampled_df, temp_df])
            
            return sampled_df
        

        # Introduce sudden drift
        # No need to random sample, bc the list will have the frequencies for each item
        random.seed(random_seed)  # For reproducibility
        drift_items_list = random.sample(items_list, k=n_items_to_drift)  
        # drift_items_list = [items_list[i] for i,x in enumerate(items_freq_list) if x == sudden_drift_start]
        renamed_items = {item: f'drifted_{item}' for item in drift_items_list}
        non_drift_items_list = list(set(items_list) - set(drift_items_list))
        
        print('drift_items_list', drift_items_list)
        print('renamed_items', renamed_items)
        print('non_drift_items_list', non_drift_items_list)

        
        random.seed(random_seed)  # For reproducibility
        sampled_df = sample_with_repetition_of_pattern(users_list,
                                                       non_drift_items_list,
                                                       non_drift_items_freq_list)
        
        sampled_df = pd.concat([sampled_df,
                                sample_with_repetition_of_pattern(users_list,
                                                                    drift_items_list,
                                                                    drift_items_freq_list)])

        if sampled_df.user_id.nunique() < n_users:
            # print(sampled_df.head())
            users_not_sampled = list(set(users_list) - set(sampled_df.user_id))
            print('users_not_sampled', len(users_not_sampled))
            # print('drift_items_list', drift_items_list)
            for user in users_not_sampled:
                for item in drift_items_list:
                    # print(sampled_df.loc[sampled_df['user_id']==user, 'item_id'].count())
                    sampled_df.loc[len(sampled_df)] = [user, item]


        sampled_df['item_id'] = sampled_df.apply(rename_item, axis=1)
        # print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))

        sampled_df['timestamp'] = ts


        sparsity, specs_str = calculate_sparsity(sampled_df)
        print('sparsity: ',sparsity)
        # print(specs_str)
        print(sampled_df.item_id.groupby([sampled_df.user_id, sampled_df.item_id]).count().unstack().fillna(0).astype(int))
        # print(sampled_df.head())


        # when trainning on pt1, yield ValueError: Some users have interacted with all items, which we can not sample negative items for them. Please set `user_inter_num_interval` to filter those users.
        # sampled_df = add_zero_user(sampled_df) # to solve the error 
        users_list.insert(0, 'u_0')

        # save_dataset_atomic_file(sampled_df, save_path, specs_str)
        split_dataset_into_4_add_users_and_save_atomic_file(sampled_df, users_list, bin_size, save_path, base_filename, specs_str)
        save_items_frequencies(n_items_to_drift, sudden_drift_start, drift_items_freq_list, non_drift_items_freq_list, save_path, base_filename, specs_str)


        return sampled_df






# Parameters
n_users = 4000 # bc of random sample, some users dont have occurrencies
n_items = 5
sudden_drift_start = 2000  # Starting user index for drift (1-indexed)
bin_size = 1000

string = "24/12/2024 21:12:24"
ts = time.mktime(datetime.strptime(string, "%d/%m/%Y %H:%M:%S").timetuple())

all_items_seen = False

random_seed = 42

ratio_to_drift = 2 # Select 50% of items to rename
n_items_to_drift = n_items // ratio_to_drift

base_filename = 'sudden_drift_dataset_all_users_start_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

drift_items_freq_list = [sudden_drift_start//2, sudden_drift_start//2]
non_drift_items_freq_list = [sudden_drift_start//4, sudden_drift_start//4, sudden_drift_start//4]

df = generate_artificial_random_dataset(n_users=n_users,
                                    n_items=n_items, 
                                    ts=ts,
                                    all_items_seen=all_items_seen,
                                    n_items_to_drift=n_items_to_drift,
                                    random_seed=random_seed,
                                    sudden_drift_start=sudden_drift_start,
                                    drift_items_freq_list=drift_items_freq_list,
                                    non_drift_items_freq_list=non_drift_items_freq_list,
                                    save_path=save_path,
                                    base_filename=base_filename,
                                    bin_size=bin_size)

drift_items_list ['i_1', 'i_5']
renamed_items {'i_1': 'drifted_i_1', 'i_5': 'drifted_i_5'}
non_drift_items_list ['i_2', 'i_4', 'i_3']
users_not_sampled 571
specs_str 4000x7_0.71
sparsity:  0.7092142857142858
item_id  drifted_i_1  drifted_i_5  i_1  i_2  i_3  i_4  i_5
user_id                                                   
u_1                0            0    1    0    0    0    0
u_10               0            0    1    0    0    0    1
u_100              0            0    0    0    0    0    1
u_1000             0            0    1    0    0    1    0
u_1001             0            0    1    1    0    1    1
...              ...          ...  ...  ...  ...  ...  ...
u_995              0            0    1    1    0    0    0
u_996              0            0    1    1    1    0    0
u_997              0            0    1    1    0    0    0
u_998              0            0    1    0    0    0    1
u_999              0            0    1    0    0    1    0

[4000 rows x 7 columns]


In [48]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-14-2025_17-34-11', data_ver='_pt2')

14 Jan 17:36    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = ERROR
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_all_users_start_i1i5_drift_all_parts_4000x7_0.71_pt1
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_all_users_start_i1i5_drift_all_parts_4000x7_0.71_pt1
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'test_only'}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'pop001', 'test': 'pop001'}}
repeatable = F



sudden_drift_dataset_all_users_start_i1i5_drift_all_parts_4000x7_0.71_pt2


14 Jan 17:36    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 2020
state = ERROR
reproducibility = True
data_path = processed_datasets/artificial_data/sudden_drift_dataset_all_users_start_i1i5_drift_all_parts_4000x7_0.71_pt2
checkpoint_dir = processed_datasets/artificial_data/sudden_drift_dataset_all_users_start_i1i5_drift_all_parts_4000x7_0.71_pt1
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'test_only'}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'pop001', 'test': 'pop001'}}
repeatable = F

User ID Range: 1 to 2000
Embedding Weight Shape: torch.Size([4001, 64])
OrderedDict([('recall@3', 1.0), ('mrr@3', 0.7271), ('ndcg@3', 0.7986), ('hit@3', 1.0), ('precision@3', 0.3333)])


# editing recbole to print train set and test set user ids

In [22]:
def trainset_testset_differs(train_data, test_data):
    # Extract user IDs from the training and test datasets
    # train_set_user_id_list = train_data.inter_feat['user_id'].tolist()
    # test_set_user_id_list = test_data.inter_feat['user_id'].tolist()

    train_set_user_id_list = train_data.dataset.inter_feat['user_id'].tolist()
    test_set_user_id_list = test_data.dataset.inter_feat['user_id'].tolist()

    print("Users in train set:",
        len(train_set_user_id_list))
    print("Users in test set:",
        len(test_set_user_id_list))
    print("Users in train set but not in test set:",
        len(list(set(train_set_user_id_list) - set(test_set_user_id_list))))
    print("Users in test set but not in train set:",
        len(list(set(test_set_user_id_list) - set(train_set_user_id_list))))


def test_on_earlier_data(model_name,
                         earlier_datasets,
                         current_dataset_name,
                         parameter_dict,
                         checkpoint_file):


    current_config,\
        current_logger,\
                current_dataset,\
                    current_train_data, current_valid_data, current_test_data = setup_config_and_dataset(model_name,
                                                                                current_dataset_name,
                                                                                parameter_dict)

    print('\nCurrent data')
    trainset_testset_differs(current_train_data, current_test_data)


    # model loading and initialization
    current_model = BPR(current_config, current_train_data.dataset).to(current_config['device'])
    current_logger.info(current_model)


    # trainer loading and initialization
    trainer = get_trainer(current_config['MODEL_TYPE'], current_config['model'])(current_config, current_model)


    # results = []

    for earlier_dataset_name in earlier_datasets:
        print('\n\n'+earlier_dataset_name)
        earlier_config,\
        earlier_logger,\
                earlier_dataset,\
                    earlier_train_data, earlier_valid_data, earlier_test_data = setup_config_and_dataset(model_name,
                                                                                earlier_dataset_name,
                                                                                parameter_dict)


        print('Earlier data')
        trainset_testset_differs(earlier_train_data, earlier_test_data)


        print('Train set: current data; Test set: earlier data')
        trainset_testset_differs(current_train_data, earlier_test_data)


        # When calculate ItemCoverage metrics, we need to run this code for set item_nums in eval_collector.
        # trainer.eval_collector.data_collect(earlier_train_data)
        trainer.eval_collector.data_collect(current_train_data)

        # model evaluation
        test_result = trainer.evaluate(earlier_test_data, model_file=checkpoint_file)
        # results += [test_result]
    
        print(test_result)


def trigger_error(data_path, base_dataset_name, model_ver, checkpoint_ver, data_ver):
    current_ver = model_ver
    current_dataset_name = base_dataset_name+current_ver

    earlier_datasets = [base_dataset_name+data_ver]#,base_dataset_name+'_pt6', base_dataset_name+'_pt7', base_dataset_name+'_pt8'] # 1,5,6,7,8


    # Checkpoint - 
    # checkpoint_ver = 'BPR-Jan-01-2025_16-20-32'
    checkpoint_dir = data_path+base_dataset_name+current_ver
    checkpoint_file = checkpoint_dir+'/'+checkpoint_ver+'.pth'


    parameter_dict = {
        'dataset': current_dataset_name+'.inter',
        'data_path': data_path,
        'load_col': {'inter': ['user_id', 'item_id']},
        'use_gpu':USE_GPU,
        'topk':K,
        'valid_metric':VALID_METRIC,
        # 'user_inter_num_interval':'[1,inf)',
        'checkpoint_dir':checkpoint_dir,
        'seed':SEED,
        'shuffle': SHUFFLE,
        'state':'ERROR',
        # 'show_progress': SHOW_PROGRESS,
        # https://recbole.io/docs/user_guide/config/evaluation_settings.html
        'eval_args': {'split': {'LS': 'test_only'}, # leave-one-out sample type
                    'group_by': 'user',
                    'order': 'RO', # random ordering
                    'mode': 'pop001'} #  for example pop100, means sample 100 negative items for each positive item in testing set based on item popularity (Counter(item) in .inter file), and evaluate the model on these positive items with their sampled negative items
    }

    test_on_earlier_data(MODEL,
                    earlier_datasets,
                    current_dataset_name,
                    parameter_dict, 
                    checkpoint_file)

In [23]:
base_filename = 'sudden_drift_dataset_u0_i1i5_drift_all_parts'
save_path = 'processed_datasets/artificial_data/'

# Current model~~~~
base_dataset_name = base_filename+'_4000x7_0.71'
data_path = 'processed_datasets/artificial_data/'

### trigger_error(model_ver='_pt1', data_ver='_pt2') (IoR)

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt1', checkpoint_ver = 'BPR-Jan-10-2025_14-26-30', data_ver='_pt2')

### trigger_error(model_ver='_pt2', data_ver='_pt5')

In [ ]:
trigger_error(data_path=data_path, base_dataset_name=base_dataset_name, model_ver='_pt2', checkpoint_ver = 'BPR-Jan-10-2025_12-00-43', data_ver='_pt5')
